**NLP**

*CC6205-1 - Otoño 2022*

Autor: Felipe Urrutia Vargas

In [1]:
# !pip install --upgrade pandas

import pandas as pd
import numpy as np
# pd.set_option("max_rows", None)
import pickle
from string import punctuation
import re

from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
# from astropy.visualization import hist
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import random

import plotly.express as px

In [2]:
sentiments = "anger fear joy sadness".split()
intensities = "low medium high".split()

In [6]:
df_train = pickle.load(open("df_train.pickle", "rb"))

In [185]:
#import spacy library
import spacy

!python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [186]:
import collections

In [187]:
!pip install unidecode
from unidecode import unidecode

!pip install Levenshtein
import Levenshtein as lev

In [188]:
import emojilib

In [189]:
vowel = "aeiou"
digit = "0123456789"

def replace_multiple(string, list_replace, replace_ch):
    for ch in list_replace:
        if ch in string:
            string = string.replace(ch, replace_ch)
    return string

def sim_lev(a, b):
    return 1 - lev.distance(a, b) / max(len(a), len(b)) if len(a) != 0 else 0

In [190]:
def get_retro_attrib(tweet):
    o = {}
    o["retro<&>num_tokens"] = len(tweet.split())
    o["retro<&>lenght"] = len(" ".join(tweet.split()))
    o["retro<&>num_numbs"] = len(re.findall(r"\d+",tweet))
    o["retro<&>num_alpha"] = len(re.findall(r"\w+", tweet))
    o["retro<&>num_with_uppercase"] = len(re.findall(r"\S*[A-Z]+\S*", tweet))
    o["retro<&>num_tokens_upper"] = sum(int(t.isupper()) for t in tweet.split())
    
    def prop_vowels(w):
        N = len(w.replace(" ", ""))
        if N>0:
            return len(re.findall(r"[aeiou]", tweet)) / N
        else:
            return 0
    
    def len_max_rep_char(w):
        w=w+" "
        c0 = w[0]
        lens = [0]
        clen = 1
        for c in w[1:]:
            if c == c0:
                clen += 1
            else:
                if c0.isalpha():
                    lens.append(clen)
                c0 = c
                clen = 1
        return max(lens)  
    
    o["retro<&>prop_vowels"] = prop_vowels(tweet.lower())
    o["retro<&>len_max_rep_char"] = len_max_rep_char(tweet.lower())
    
    def max_char_fre_per_token(w, c="k"):
        tw = w.split()
        fmax = 0
        for t in tw:
            f = sum(int(ch==c) for ch in t)
            if f>fmax:
                fmax = f
        return fmax
    
    o["retro<&>max_char_fre_per_token(o)"] = max_char_fre_per_token(tweet.lower(), c="o")
    o["retro<&>max_char_fre_per_token(s)"] = max_char_fre_per_token(tweet.lower(), c="s")
    o["retro<&>max_char_fre_per_token(g)"] = max_char_fre_per_token(tweet.lower(), c="g")    
    o["retro<&>max_char_fre_per_token(l)"] = max_char_fre_per_token(tweet.lower(), c="l")    
    
    def max_type_rep_char_per_token(w, t="vowel"):
        w=unidecode(w+" ")
        c0 = w[0]
        lens = [0]
        clen = 1
        for c in w[1:]:
            if (c0.isalpha() and c.isalpha()) and ((c in "aeiou" and c0 in "aeiou") or (c not in "aeiou" and c0 not in "aeiou")):
                clen += 1
            else:
                if t=="vowel":
                    if c0 in "aeiou":
                        lens.append(clen)
                else:
                    if c0 not in "aeiou":
                        lens.append(clen) 
                c0 = c
                clen = 1
        return max(lens) 
    
    o["retro<&>max_type_rep_char_per_token(vowel)"] = max_type_rep_char_per_token(tweet.lower(), t="vowel")
    
    return o

In [191]:
def get_punct_attrib(tweet):
    o = {}
    o["punct<&>[\.]{3}"] = len(re.findall(r"[\.]{3}", tweet))
    o["punct<&>[!]"] = len(re.findall(r"[!]", tweet))
    o["punct<&>[#]"] = len(re.findall(r"[#]", tweet))
    o["punct<&>[#]{1}\S+"] = len(re.findall(r"[#]{1}\S+", tweet))
    o["punct<&>[\*]"] = len(re.findall(r"[\*]", tweet))
    o["punct<&>[@]{1}\S+"] = len(re.findall(r"[@]{1}\S+", tweet))
    o["punct<&>\S*[?]{1}\S*"] = len(re.findall(r"\S*[?]{1}\S*", tweet))
    return o

In [192]:
def get_emojilib_attrib(tweet):
    emo_list = emojilib.emoji_list(tweet)
    emo_names = list([d['name'] for d in emo_list if 'name' in d])
    o = {}
    for emo in emo_names:
        if emo not in o.keys():
            o["emoji<&>"+emo] = 0
        o["emoji<&>"+emo] += 1
    return o

In [193]:
def get_linguistics_attrib(tweet):
    o = {}
    nlp_tweet = nlp(tweet)
    for token in nlp_tweet:
        label = "text lemma pos tag dep shape is_alpha is_stop".split()
        vals = [token.text, token.lemma_.lower(), token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop]
        dict_vals = dict(zip(label[1:], vals[1:]))
        for k, v in dict_vals.items():
            l = f"linguistics<&>{k}<&>{v}"
            if l not in o.keys():
                o[l] = 0
            o[l] += 1
    return o    

In [202]:
get_retro_attrib(tweet), get_punct_attrib(tweet), get_emojilib_attrib(tweet), get_linguistics_attrib(tweet)

({'retro<&>num_tokens': 22,
  'retro<&>lenght': 126,
  'retro<&>num_numbs': 0,
  'retro<&>num_alpha': 22,
  'retro<&>num_with_uppercase': 5,
  'retro<&>num_tokens_upper': 1,
  'retro<&>prop_vowels': 0.29523809523809524,
  'retro<&>len_max_rep_char': 2,
  'retro<&>max_char_fre_per_token(o)': 1,
  'retro<&>max_char_fre_per_token(s)': 3,
  'retro<&>max_char_fre_per_token(g)': 1,
  'retro<&>max_char_fre_per_token(l)': 1,
  'retro<&>max_type_rep_char_per_token(vowel)': 2},
 {'punct<&>[\\.]{3}': 0,
  'punct<&>[!]': 1,
  'punct<&>[#]': 0,
  'punct<&>[#]{1}\\S+': 0,
  'punct<&>[\\*]': 0,
  'punct<&>[@]{1}\\S+': 1,
  'punct<&>\\S*[?]{1}\\S*': 1},
 {},
 {'linguistics<&>lemma<&>@sainsburys': 1,
  'linguistics<&>pos<&>PROPN': 2,
  'linguistics<&>tag<&>NNP': 2,
  'linguistics<&>dep<&>dep': 1,
  'linguistics<&>shape<&>@xxxx': 1,
  'linguistics<&>is_alpha<&>False': 4,
  'linguistics<&>is_stop<&>False': 16,
  'linguistics<&>lemma<&>could': 1,
  'linguistics<&>pos<&>AUX': 1,
  'linguistics<&>tag<&>MD':

In [195]:
import time
import datetime

In [22]:
data_all_attributes = []
times = []
for k, ix in enumerate(df_train.index):
    start_time = time.time()
    tweet = df_train.loc[ix]["text"]
    o = {"id": ix}
    o = {**o, **get_retro_attrib(tweet)}
    o = {**o, **get_punct_attrib(tweet)}
    o = {**o, **get_emojilib_attrib(tweet)}
    o = {**o, **get_linguistics_attrib(tweet)}
    data_all_attributes.append(o)
    dt = time.time()-start_time
    times.append(dt)
    print(f"""
    row: {k+1}, total_rows: {df_train.shape[0]}
    progress: {np.round(100*(k+1)/df_train.shape[0], 3)}%
    wait time: {datetime.timedelta(seconds = np.mean(times)*(df_train.shape[0]-k-1))}s
    """)


    row: 1, total_rows: 3960
    progress: 0.025%
    wait time: 0:00:31.703685s
    

    row: 2, total_rows: 3960
    progress: 0.051%
    wait time: 0:00:33.673118s
    

    row: 3, total_rows: 3960
    progress: 0.076%
    wait time: 0:00:30.365305s
    

    row: 4, total_rows: 3960
    progress: 0.101%
    wait time: 0:00:30.686960s
    

    row: 5, total_rows: 3960
    progress: 0.126%
    wait time: 0:00:29.293544s
    

    row: 6, total_rows: 3960
    progress: 0.152%
    wait time: 0:00:27.043909s
    

    row: 7, total_rows: 3960
    progress: 0.177%
    wait time: 0:00:27.696191s
    

    row: 8, total_rows: 3960
    progress: 0.202%
    wait time: 0:00:27.689202s
    

    row: 9, total_rows: 3960
    progress: 0.227%
    wait time: 0:00:26.803435s
    

    row: 10, total_rows: 3960
    progress: 0.253%
    wait time: 0:00:26.489263s
    

    row: 11, total_rows: 3960
    progress: 0.278%
    wait time: 0:00:25.871547s
    

    row: 12, total_rows: 3960
    progre


    row: 100, total_rows: 3960
    progress: 2.525%
    wait time: 0:00:22.041763s
    

    row: 101, total_rows: 3960
    progress: 2.551%
    wait time: 0:00:22.047333s
    

    row: 102, total_rows: 3960
    progress: 2.576%
    wait time: 0:00:21.976989s
    

    row: 103, total_rows: 3960
    progress: 2.601%
    wait time: 0:00:22.020336s
    

    row: 104, total_rows: 3960
    progress: 2.626%
    wait time: 0:00:21.988496s
    

    row: 105, total_rows: 3960
    progress: 2.652%
    wait time: 0:00:21.920411s
    

    row: 106, total_rows: 3960
    progress: 2.677%
    wait time: 0:00:21.926326s
    

    row: 107, total_rows: 3960
    progress: 2.702%
    wait time: 0:00:21.859961s
    

    row: 108, total_rows: 3960
    progress: 2.727%
    wait time: 0:00:21.830415s
    

    row: 109, total_rows: 3960
    progress: 2.753%
    wait time: 0:00:21.765966s
    

    row: 110, total_rows: 3960
    progress: 2.778%
    wait time: 0:00:21.737639s
    

    row: 111, total_


    row: 203, total_rows: 3960
    progress: 5.126%
    wait time: 0:00:20.756598s
    

    row: 204, total_rows: 3960
    progress: 5.152%
    wait time: 0:00:20.796785s
    

    row: 205, total_rows: 3960
    progress: 5.177%
    wait time: 0:00:20.781497s
    

    row: 206, total_rows: 3960
    progress: 5.202%
    wait time: 0:00:20.784567s
    

    row: 207, total_rows: 3960
    progress: 5.227%
    wait time: 0:00:20.823815s
    

    row: 208, total_rows: 3960
    progress: 5.253%
    wait time: 0:00:20.808442s
    

    row: 209, total_rows: 3960
    progress: 5.278%
    wait time: 0:00:20.757254s
    

    row: 210, total_rows: 3960
    progress: 5.303%
    wait time: 0:00:20.742267s
    

    row: 211, total_rows: 3960
    progress: 5.328%
    wait time: 0:00:20.727383s
    

    row: 212, total_rows: 3960
    progress: 5.354%
    wait time: 0:00:20.730275s
    

    row: 213, total_rows: 3960
    progress: 5.379%
    wait time: 0:00:20.750694s
    

    row: 214, total_


    row: 301, total_rows: 3960
    progress: 7.601%
    wait time: 0:00:20.258691s
    

    row: 302, total_rows: 3960
    progress: 7.626%
    wait time: 0:00:20.270951s
    

    row: 303, total_rows: 3960
    progress: 7.652%
    wait time: 0:00:20.258935s
    

    row: 304, total_rows: 3960
    progress: 7.677%
    wait time: 0:00:20.258991s
    

    row: 305, total_rows: 3960
    progress: 7.702%
    wait time: 0:00:20.259010s
    

    row: 306, total_rows: 3960
    progress: 7.727%
    wait time: 0:00:20.247044s
    

    row: 307, total_rows: 3960
    progress: 7.753%
    wait time: 0:00:20.235111s
    

    row: 308, total_rows: 3960
    progress: 7.778%
    wait time: 0:00:20.223243s
    

    row: 309, total_rows: 3960
    progress: 7.803%
    wait time: 0:00:20.199572s
    

    row: 310, total_rows: 3960
    progress: 7.828%
    wait time: 0:00:20.187823s
    

    row: 311, total_rows: 3960
    progress: 7.854%
    wait time: 0:00:20.187858s
    

    row: 312, total_


    row: 401, total_rows: 3960
    progress: 10.126%
    wait time: 0:00:19.690361s
    

    row: 402, total_rows: 3960
    progress: 10.152%
    wait time: 0:00:19.697875s
    

    row: 403, total_rows: 3960
    progress: 10.177%
    wait time: 0:00:19.678808s
    

    row: 404, total_rows: 3960
    progress: 10.202%
    wait time: 0:00:19.677442s
    

    row: 405, total_rows: 3960
    progress: 10.227%
    wait time: 0:00:19.684835s
    

    row: 406, total_rows: 3960
    progress: 10.253%
    wait time: 0:00:19.674636s
    

    row: 407, total_rows: 3960
    progress: 10.278%
    wait time: 0:00:19.673199s
    

    row: 408, total_rows: 3960
    progress: 10.303%
    wait time: 0:00:19.671736s
    

    row: 409, total_rows: 3960
    progress: 10.328%
    wait time: 0:00:19.644186s
    

    row: 410, total_rows: 3960
    progress: 10.354%
    wait time: 0:00:19.625426s
    

    row: 411, total_rows: 3960
    progress: 10.379%
    wait time: 0:00:19.615365s
    

    row: 

...


    row: 3497, total_rows: 3960
    progress: 88.308%
    wait time: 0:00:02.482004s
    

    row: 3498, total_rows: 3960
    progress: 88.333%
    wait time: 0:00:02.476597s
    

    row: 3499, total_rows: 3960
    progress: 88.359%
    wait time: 0:00:02.471321s
    

    row: 3500, total_rows: 3960
    progress: 88.384%
    wait time: 0:00:02.466176s
    

    row: 3501, total_rows: 3960
    progress: 88.409%
    wait time: 0:00:02.460900s
    

    row: 3502, total_rows: 3960
    progress: 88.434%
    wait time: 0:00:02.455622s
    

    row: 3503, total_rows: 3960
    progress: 88.46%
    wait time: 0:00:02.450084s
    

    row: 3504, total_rows: 3960
    progress: 88.485%
    wait time: 0:00:02.444676s
    

    row: 3505, total_rows: 3960
    progress: 88.51%
    wait time: 0:00:02.439269s
    

    row: 3506, total_rows: 3960
    progress: 88.535%
    wait time: 0:00:02.433991s
    

    row: 3507, total_rows: 3960
    progress: 88.561%
    wait time: 0:00:02.428713s
    




    row: 3590, total_rows: 3960
    progress: 90.657%
    wait time: 0:00:01.986854s
    

    row: 3591, total_rows: 3960
    progress: 90.682%
    wait time: 0:00:01.981549s
    

    row: 3592, total_rows: 3960
    progress: 90.707%
    wait time: 0:00:01.976142s
    

    row: 3593, total_rows: 3960
    progress: 90.732%
    wait time: 0:00:01.970837s
    

    row: 3594, total_rows: 3960
    progress: 90.758%
    wait time: 0:00:01.965531s
    

    row: 3595, total_rows: 3960
    progress: 90.783%
    wait time: 0:00:01.960226s
    

    row: 3596, total_rows: 3960
    progress: 90.808%
    wait time: 0:00:01.954919s
    

    row: 3597, total_rows: 3960
    progress: 90.833%
    wait time: 0:00:01.949512s
    

    row: 3598, total_rows: 3960
    progress: 90.859%
    wait time: 0:00:01.944205s
    

    row: 3599, total_rows: 3960
    progress: 90.884%
    wait time: 0:00:01.938697s
    

    row: 3600, total_rows: 3960
    progress: 90.909%
    wait time: 0:00:01.933491s
    


    row: 3684, total_rows: 3960
    progress: 93.03%
    wait time: 0:00:01.483562s
    

    row: 3685, total_rows: 3960
    progress: 93.056%
    wait time: 0:00:01.478309s
    

    row: 3686, total_rows: 3960
    progress: 93.081%
    wait time: 0:00:01.472831s
    

    row: 3687, total_rows: 3960
    progress: 93.106%
    wait time: 0:00:01.467577s
    

    row: 3688, total_rows: 3960
    progress: 93.131%
    wait time: 0:00:01.462247s
    

    row: 3689, total_rows: 3960
    progress: 93.157%
    wait time: 0:00:01.456918s
    

    row: 3690, total_rows: 3960
    progress: 93.182%
    wait time: 0:00:01.451514s
    

    row: 3691, total_rows: 3960
    progress: 93.207%
    wait time: 0:00:01.446038s
    

    row: 3692, total_rows: 3960
    progress: 93.232%
    wait time: 0:00:01.440709s
    

    row: 3693, total_rows: 3960
    progress: 93.258%
    wait time: 0:00:01.435306s
    

    row: 3694, total_rows: 3960
    progress: 93.283%
    wait time: 0:00:01.430048s
    



    row: 3779, total_rows: 3960
    progress: 95.429%
    wait time: 0:00:00.973339s
    

    row: 3780, total_rows: 3960
    progress: 95.455%
    wait time: 0:00:00.968039s
    

    row: 3781, total_rows: 3960
    progress: 95.48%
    wait time: 0:00:00.962596s
    

    row: 3782, total_rows: 3960
    progress: 95.505%
    wait time: 0:00:00.957248s
    

    row: 3783, total_rows: 3960
    progress: 95.53%
    wait time: 0:00:00.951806s
    

    row: 3784, total_rows: 3960
    progress: 95.556%
    wait time: 0:00:00.946411s
    

    row: 3785, total_rows: 3960
    progress: 95.581%
    wait time: 0:00:00.941109s
    

    row: 3786, total_rows: 3960
    progress: 95.606%
    wait time: 0:00:00.935760s
    

    row: 3787, total_rows: 3960
    progress: 95.631%
    wait time: 0:00:00.930365s
    

    row: 3788, total_rows: 3960
    progress: 95.657%
    wait time: 0:00:00.925015s
    

    row: 3789, total_rows: 3960
    progress: 95.682%
    wait time: 0:00:00.919575s
    




    row: 3874, total_rows: 3960
    progress: 97.828%
    wait time: 0:00:00.462795s
    

    row: 3875, total_rows: 3960
    progress: 97.854%
    wait time: 0:00:00.457405s
    

    row: 3876, total_rows: 3960
    progress: 97.879%
    wait time: 0:00:00.451972s
    

    row: 3877, total_rows: 3960
    progress: 97.904%
    wait time: 0:00:00.446669s
    

    row: 3878, total_rows: 3960
    progress: 97.929%
    wait time: 0:00:00.441280s
    

    row: 3879, total_rows: 3960
    progress: 97.955%
    wait time: 0:00:00.435932s
    

    row: 3880, total_rows: 3960
    progress: 97.98%
    wait time: 0:00:00.430563s
    

    row: 3881, total_rows: 3960
    progress: 98.005%
    wait time: 0:00:00.425194s
    

    row: 3882, total_rows: 3960
    progress: 98.03%
    wait time: 0:00:00.419824s
    

    row: 3883, total_rows: 3960
    progress: 98.056%
    wait time: 0:00:00.414414s
    

    row: 3884, total_rows: 3960
    progress: 98.081%
    wait time: 0:00:00.409045s
    



In [23]:
df_representation = pd.DataFrame(data_all_attributes).set_index("id").fillna(0)
df_representation

,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,retro<&>num_tokens_upper,retro<&>prop_vowels,retro<&>len_max_rep_char,retro<&>max_char_fre_per_token(o),retro<&>max_char_fre_per_token(s),...,linguistics<&>lemma<&>scritche,"linguistics<&>lemma<&>loud,\n#amndbot",linguistics<&>lemma<&>randomly,linguistics<&>lemma<&>essay,linguistics<&>lemma<&>concession,linguistics<&>lemma<&>shortsightedness,linguistics<&>lemma<&>@keitholbermann,linguistics<&>lemma<&>despicable,linguistics<&>lemma<&>rancour,linguistics<&>lemma<&>immoral
id,,,,,,,,,,,,,,,,,,,,,
10000,18,96,0,18,3,1,0.253165,2,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,23,119,0,25,7,2,0.329897,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,19,108,0,19,2,2,0.333333,2,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,24,134,0,23,1,1,0.315315,2,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,24,125,0,25,3,0,0.352941,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40855,24,112,0,23,2,1,0.325843,2,2,2,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
40856,20,121,0,21,1,0,0.352941,2,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40857,23,139,0,23,2,1,0.316239,2,2,4,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [3]:
# pickle.dump(df_representation, open("df_representation_v1.pickle", "wb"))
df_representation = pickle.load(open("df_representation_v1.pickle", "rb"))

In [203]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
indexs = df_train[df_train["sen"] == sentiments[3]].index
indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin(["low", "high"]))].index

X = df_representation.loc[indexsLH]#[rel_cols]
y = df_train.loc[X.index]["int"]

dic_label_count = y.value_counts().to_dict()
min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
index_label_1 = y[y==min_label].index
oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
X_res, y_res = X.copy(), y.copy()

for step in range(oversampling_steps):
    new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
    copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
    copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
    X_res = pd.concat([X_res, copied_sub_X], axis=0)
    y_res = pd.concat([y_res, copied_sub_y], axis=0)

X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
selector = SelectKBest(chi2, k=X.shape[1])
X_res_ = X_res - X_res.min()
selector.fit(X_res_, y_res)
scores_selector = {col: selector.scores_[i] if str(selector.scores_[i]) != "nan" else 0 for i, col in enumerate(X.columns.tolist())}
ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: y[1], reverse=True)]
ranked_cols


In [20]:
indexs = df_train[df_train["sen"] == sentiments[3]].index
scores_selector = {col: [] for col in df_representation.columns.tolist()}
comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
for l1 in intensities:
    for l2 in intensities:
        if l1 < l2:
            indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

            X = df_representation.loc[indexsLH]#[rel_cols]
            y = df_train.loc[X.index]["int"]

            dic_label_count = y.value_counts().to_dict()
            min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
            max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
            index_label_1 = y[y==min_label].index
            oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
            X_res, y_res = X.copy(), y.copy()

            for step in range(oversampling_steps):
                new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                X_res = pd.concat([X_res, copied_sub_X], axis=0)
                y_res = pd.concat([y_res, copied_sub_y], axis=0)

            X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
            selector = SelectKBest(chi2, k=X.shape[1])
            X_res_ = X_res - X_res.min()
            selector.fit(X_res_, y_res)
            for i, col in enumerate(X.columns.tolist()):
                scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]
comp_l, ranked_cols

In [28]:
ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]
comp_l, ranked_cols

([('low', 'medium'), ('high', 'low'), ('high', 'medium')],
 ['linguistics<&>lemma<&>depress',
  'linguistics<&>lemma<&>depressing',
  'linguistics<&>lemma<&>grim',
  'linguistics<&>lemma<&>depression',
  'linguistics<&>lemma<&>sadness',
  'linguistics<&>lemma<&>honestly',
  'linguistics<&>lemma<&>️',
  'linguistics<&>shape<&>️',
  'linguistics<&>tag<&>-RRB-',
  'linguistics<&>lemma<&>frown',
  'linguistics<&>lemma<&>)',
  'linguistics<&>shape<&>)',
  'linguistics<&>tag<&>-LRB-',
  'linguistics<&>lemma<&>either',
  'linguistics<&>lemma<&>sadly',
  'linguistics<&>lemma<&>true',
  'linguistics<&>lemma<&>(',
  'linguistics<&>shape<&>(',
  'linguistics<&>lemma<&>something',
  'linguistics<&>lemma<&>sad',
  'linguistics<&>lemma<&>blue',
  'emoji<&>_face_with_tears_of_joy_',
  'linguistics<&>lemma<&>soul',
  'linguistics<&>lemma<&>fucking',
  'linguistics<&>lemma<&>tired',
  'emoji<&>_disappointed_face_',
  'linguistics<&>lemma<&>😞',
  'linguistics<&>shape<&>😞',
  'linguistics<&>lemma<&>body'

In [204]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support

In [50]:
sen = sentiments[0]

# indexs = df_train[df_train["sen"] == sen].index
# indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin(["low", "high"]))].index

# X = df_representation.loc[indexsLH]#[rel_cols]
# y = df_train.loc[X.index]["int"]

# dic_label_count = y.value_counts().to_dict()
# min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
# max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
# index_label_1 = y[y==min_label].index
# oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
# X_res, y_res = X.copy(), y.copy()

# for step in range(oversampling_steps):
#     new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
#     copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
#     copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
#     X_res = pd.concat([X_res, copied_sub_X], axis=0)
#     y_res = pd.concat([y_res, copied_sub_y], axis=0)

# X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
# selector = SelectKBest(chi2, k=X.shape[1])
# X_res_ = X_res - X_res.min()
# selector.fit(X_res_, y_res)
# scores_selector = {col: selector.scores_[i] if str(selector.scores_[i]) != "nan" else 0 for i, col in enumerate(X.columns.tolist())}
# ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda x: x[1], reverse=True)]
# ranked_cols

indexs = df_train[df_train["sen"] == sen].index
scores_selector = {col: [] for col in df_representation.columns.tolist()}
comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
for l1 in intensities:
    for l2 in intensities:
        if l1 < l2:
            indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

            X = df_representation.loc[indexsLH]#[rel_cols]
            y = df_train.loc[X.index]["int"]

            dic_label_count = y.value_counts().to_dict()
            min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
            max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
            index_label_1 = y[y==min_label].index
            oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
            X_res, y_res = X.copy(), y.copy()

            for step in range(oversampling_steps):
                new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                X_res = pd.concat([X_res, copied_sub_X], axis=0)
                y_res = pd.concat([y_res, copied_sub_y], axis=0)

            X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
            selector = SelectKBest(chi2, k=X.shape[1])
            X_res_ = X_res - X_res.min()
            selector.fit(X_res_, y_res)
            for i, col in enumerate(X.columns.tolist()):
                scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]
# comp_l, ranked_cols

f1_weight = []

for num_cols in range(1, len(ranked_cols), 10):
    X = df_representation.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(f1_weight[-1])

[1, 0.10424886540056362, 0.019309878014991615]
[11, 0.6411688350633156, 0.029228674134960844]
[21, 0.6609119066959994, 0.02203836650268399]
[31, 0.6588559737273805, 0.012008818616047677]
[41, 0.66215257900806, 0.004897267596503262]
[51, 0.6819780207011394, 0.018291375701466977]
[61, 0.6891776153090093, 0.018405652049383794]
[71, 0.6948351734809612, 0.026925106785193546]
[81, 0.698515744491627, 0.030620007364956892]
[91, 0.6995012251275626, 0.029302400431879483]
[101, 0.7001325185391131, 0.029258873024053687]
[111, 0.7115093975305629, 0.02715503116164831]
[121, 0.710112986921863, 0.02801785329972967]
[131, 0.7000463757225261, 0.03418087393899213]
[141, 0.7062307214260791, 0.03812506665278833]
[151, 0.7079706264171842, 0.03664070574426643]
[161, 0.7036688956016317, 0.038892980889282364]
[171, 0.7010408157744935, 0.04568406926917083]
[181, 0.6933352645072454, 0.032473129834692624]
[191, 0.6942896187745584, 0.04011906590259415]
[201, 0.6928814435108482, 0.043088935779638876]
[211, 0.686060

KeyboardInterrupt: 

In [51]:
best_f1 = sorted(f1_weight, key=lambda x: x[1], reverse=True)[0][0]
best_f1

721

In [52]:
fine_f1_weight = []

for num_cols in range(best_f1-10, best_f1+10, 1):
    X = df_representation.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    fine_f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(fine_f1_weight[-1])

[711, 0.7119090707998, 0.027855017461954637]
[712, 0.7136188549235528, 0.025519407164058552]
[713, 0.7136188549235528, 0.025519407164058552]
[714, 0.7136188549235528, 0.025519407164058552]
[715, 0.7147670371058765, 0.02471609889494386]
[716, 0.7152799717726143, 0.02237807015865809]
[717, 0.7152205637645912, 0.02236798949000108]
[718, 0.7152205637645912, 0.02236798949000108]
[719, 0.7152205637645912, 0.02236798949000108]
[720, 0.7139066277577931, 0.023421575299425103]
[721, 0.7139066277577931, 0.023421575299425103]
[722, 0.7130921410103506, 0.02330435003286564]
[723, 0.7130204092882748, 0.02329945517365635]
[724, 0.7130204092882748, 0.02329945517365635]
[725, 0.7130204092882748, 0.02329945517365635]
[726, 0.7138116315671187, 0.022539456984139863]
[727, 0.7138116315671187, 0.022539456984139863]
[728, 0.7138116315671187, 0.022539456984139863]
[729, 0.7138116315671187, 0.022539456984139863]
[730, 0.7138116315671187, 0.022539456984139863]


In [53]:
fine_best_f1 = sorted(fine_f1_weight, key=lambda x: x[1], reverse=True)[0][0]
fine_best_f1#0.577

716

In [54]:
pickle.dump(ranked_cols[:fine_best_f1+1], open(f"cols_selected_{sen}_v1.pickle", "wb"))

In [56]:
sen = sentiments[0]
print(sen)
cols_selected_sen = pickle.load(open(f"cols_selected_{sen}_v3.pickle", "rb"))
print(len(cols_selected_sen))
cols_selected_sen

anger
953


['linguistics<&>lemma<&>fume',
 'linguistics<&>lemma<&>fuck',
 'linguistics<&>lemma<&>angry',
 'linguistics<&>lemma<&>snap',
 'linguistics<&>lemma<&>so',
 'linguistics<&>lemma<&>:',
 'linguistics<&>shape<&>:',
 "linguistics<&>lemma<&>'s",
 'linguistics<&>lemma<&>fucking',
 'linguistics<&>tag<&>JJR',
 'linguistics<&>lemma<&>well',
 'linguistics<&>lemma<&>relentless',
 'linguistics<&>lemma<&>frown',
 'linguistics<&>lemma<&>wtf',
 'linguistics<&>lemma<&>irritate',
 'linguistics<&>lemma<&>boil',
 'linguistics<&>lemma<&>incense',
 'linguistics<&>lemma<&>piss',
 'linguistics<&>lemma<&>blood',
 'linguistics<&>lemma<&>again',
 'linguistics<&>lemma<&>then',
 'emoji<&>_pouting_face_',
 'linguistics<&>lemma<&>outrage',
 'linguistics<&>lemma<&>why',
 'linguistics<&>lemma<&>right',
 'linguistics<&>lemma<&>offense',
 'linguistics<&>lemma<&>laugh',
 'linguistics<&>lemma<&>pic',
 'linguistics<&>lemma<&>burst',
 'linguistics<&>lemma<&>shoot',
 'linguistics<&>lemma<&>move',
 'linguistics<&>lemma<&>hell'

In [205]:
!pip install torchvision

In [206]:
!pip install transformers

In [207]:
import torch
from transformers import AutoModel, AutoTokenizer

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

# For transformers v3.x:
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
data_bertweet = []
col_names = [f"BERTweet_{i}" for i in range(768)]
for k, ix in enumerate(df_train.index):
    print(k+1, df_train.shape[0], 100*(k+1)/df_train.shape[0])
    tweet = df_train.loc[ix]["text"]
    input_ids = torch.tensor([tokenizer.encode(tweet)])
    with torch.no_grad():
        outputs = bertweet(input_ids)
        hidden_states = outputs[0]

    token_embeddings = np.array([ll.numpy() for ll in hidden_states[0]])
    sentence_embedding = np.mean(token_embeddings, axis=0)
    o = {"id": ix}
    o = {**o, **dict(zip(col_names, sentence_embedding))}
    data_bertweet.append(o)

1 3960 0.025252525252525252
2 3960 0.050505050505050504
3 3960 0.07575757575757576
4 3960 0.10101010101010101
5 3960 0.12626262626262627
6 3960 0.15151515151515152
7 3960 0.17676767676767677
8 3960 0.20202020202020202
9 3960 0.22727272727272727
10 3960 0.25252525252525254
11 3960 0.2777777777777778
12 3960 0.30303030303030304
13 3960 0.3282828282828283
14 3960 0.35353535353535354
15 3960 0.3787878787878788
16 3960 0.40404040404040403
17 3960 0.4292929292929293
18 3960 0.45454545454545453
19 3960 0.4797979797979798
20 3960 0.5050505050505051
21 3960 0.5303030303030303
22 3960 0.5555555555555556
23 3960 0.5808080808080808
24 3960 0.6060606060606061
25 3960 0.6313131313131313
26 3960 0.6565656565656566
27 3960 0.6818181818181818
28 3960 0.7070707070707071
29 3960 0.7323232323232324
30 3960 0.7575757575757576
31 3960 0.7828282828282829
32 3960 0.8080808080808081
33 3960 0.8333333333333334
34 3960 0.8585858585858586
35 3960 0.8838383838383839
36 3960 0.9090909090909091
37 3960 0.93434343434

307 3960 7.752525252525253
308 3960 7.777777777777778
309 3960 7.803030303030303
310 3960 7.828282828282828
311 3960 7.853535353535354
312 3960 7.878787878787879
313 3960 7.904040404040404
314 3960 7.929292929292929
315 3960 7.954545454545454
316 3960 7.97979797979798
317 3960 8.005050505050505
318 3960 8.030303030303031
319 3960 8.055555555555555
320 3960 8.080808080808081
321 3960 8.106060606060606
322 3960 8.131313131313131
323 3960 8.156565656565656
324 3960 8.181818181818182
325 3960 8.207070707070708
326 3960 8.232323232323232
327 3960 8.257575757575758
328 3960 8.282828282828282
329 3960 8.308080808080808
330 3960 8.333333333333334
331 3960 8.358585858585858
332 3960 8.383838383838384
333 3960 8.409090909090908
334 3960 8.434343434343434
335 3960 8.45959595959596
336 3960 8.484848484848484
337 3960 8.51010101010101
338 3960 8.535353535353535
339 3960 8.56060606060606
340 3960 8.585858585858587
341 3960 8.61111111111111
342 3960 8.636363636363637
343 3960 8.66161616161616
344 396

609 3960 15.378787878787879
610 3960 15.404040404040405
611 3960 15.429292929292929
612 3960 15.454545454545455
613 3960 15.47979797979798
614 3960 15.505050505050505
615 3960 15.530303030303031
616 3960 15.555555555555555
617 3960 15.580808080808081
618 3960 15.606060606060606
619 3960 15.631313131313131
620 3960 15.656565656565656
621 3960 15.681818181818182
622 3960 15.707070707070708
623 3960 15.732323232323232
624 3960 15.757575757575758
625 3960 15.782828282828282
626 3960 15.808080808080808
627 3960 15.833333333333334
628 3960 15.858585858585858
629 3960 15.883838383838384
630 3960 15.909090909090908
631 3960 15.934343434343434
632 3960 15.95959595959596
633 3960 15.984848484848484
634 3960 16.01010101010101
635 3960 16.035353535353536
636 3960 16.060606060606062
637 3960 16.085858585858585
638 3960 16.11111111111111
639 3960 16.136363636363637
640 3960 16.161616161616163
641 3960 16.18686868686869
642 3960 16.21212121212121
643 3960 16.237373737373737
644 3960 16.26262626262626

907 3960 22.904040404040405
908 3960 22.92929292929293
909 3960 22.954545454545453
910 3960 22.97979797979798
911 3960 23.005050505050505
912 3960 23.03030303030303
913 3960 23.055555555555557
914 3960 23.08080808080808
915 3960 23.106060606060606
916 3960 23.13131313131313
917 3960 23.156565656565657
918 3960 23.181818181818183
919 3960 23.207070707070706
920 3960 23.232323232323232
921 3960 23.257575757575758
922 3960 23.282828282828284
923 3960 23.30808080808081
924 3960 23.333333333333332
925 3960 23.358585858585858
926 3960 23.383838383838384
927 3960 23.40909090909091
928 3960 23.434343434343436
929 3960 23.45959595959596
930 3960 23.484848484848484
931 3960 23.51010101010101
932 3960 23.535353535353536
933 3960 23.560606060606062
934 3960 23.585858585858585
935 3960 23.61111111111111
936 3960 23.636363636363637
937 3960 23.661616161616163
938 3960 23.68686868686869
939 3960 23.71212121212121
940 3960 23.737373737373737
941 3960 23.762626262626263
942 3960 23.78787878787879
943 3

1199 3960 30.27777777777778
1200 3960 30.303030303030305
1201 3960 30.328282828282827
1202 3960 30.353535353535353
1203 3960 30.37878787878788
1204 3960 30.404040404040405
1205 3960 30.42929292929293
1206 3960 30.454545454545453
1207 3960 30.47979797979798
1208 3960 30.505050505050505
1209 3960 30.53030303030303
1210 3960 30.555555555555557
1211 3960 30.58080808080808
1212 3960 30.606060606060606
1213 3960 30.63131313131313
1214 3960 30.656565656565657
1215 3960 30.681818181818183
1216 3960 30.707070707070706
1217 3960 30.732323232323232
1218 3960 30.757575757575758
1219 3960 30.782828282828284
1220 3960 30.80808080808081
1221 3960 30.833333333333332
1222 3960 30.858585858585858
1223 3960 30.883838383838384
1224 3960 30.90909090909091
1225 3960 30.934343434343436
1226 3960 30.95959595959596
1227 3960 30.984848484848484
1228 3960 31.01010101010101
1229 3960 31.035353535353536
1230 3960 31.060606060606062
1231 3960 31.085858585858585
1232 3960 31.11111111111111
1233 3960 31.1363636363636

...

2670 3960 67.42424242424242
2671 3960 67.44949494949495
2672 3960 67.47474747474747
2673 3960 67.5
2674 3960 67.52525252525253
2675 3960 67.55050505050505
2676 3960 67.57575757575758
2677 3960 67.6010101010101
2678 3960 67.62626262626263
2679 3960 67.65151515151516
2680 3960 67.67676767676768
2681 3960 67.70202020202021
2682 3960 67.72727272727273
2683 3960 67.75252525252525
2684 3960 67.77777777777777
2685 3960 67.8030303030303
2686 3960 67.82828282828282
2687 3960 67.85353535353535
2688 3960 67.87878787878788
2689 3960 67.9040404040404
2690 3960 67.92929292929293
2691 3960 67.95454545454545
2692 3960 67.97979797979798
2693 3960 68.0050505050505
2694 3960 68.03030303030303
2695 3960 68.05555555555556
2696 3960 68.08080808080808
2697 3960 68.10606060606061
2698 3960 68.13131313131314
2699 3960 68.15656565656566
2700 3960 68.18181818181819
2701 3960 68.20707070707071
2702 3960 68.23232323232324
2703 3960 68.25757575757575
2704 3960 68.28282828282828
2705 3960 68.3080808080808
2706 3960 

2967 3960 74.92424242424242
2968 3960 74.94949494949495
2969 3960 74.97474747474747
2970 3960 75.0
2971 3960 75.02525252525253
2972 3960 75.05050505050505
2973 3960 75.07575757575758
2974 3960 75.1010101010101
2975 3960 75.12626262626263
2976 3960 75.15151515151516
2977 3960 75.17676767676768
2978 3960 75.20202020202021
2979 3960 75.22727272727273
2980 3960 75.25252525252525
2981 3960 75.27777777777777
2982 3960 75.3030303030303
2983 3960 75.32828282828282
2984 3960 75.35353535353535
2985 3960 75.37878787878788
2986 3960 75.4040404040404
2987 3960 75.42929292929293
2988 3960 75.45454545454545
2989 3960 75.47979797979798
2990 3960 75.5050505050505
2991 3960 75.53030303030303
2992 3960 75.55555555555556
2993 3960 75.58080808080808
2994 3960 75.60606060606061
2995 3960 75.63131313131314
2996 3960 75.65656565656566
2997 3960 75.68181818181819
2998 3960 75.70707070707071
2999 3960 75.73232323232324
3000 3960 75.75757575757575
3001 3960 75.78282828282828
3002 3960 75.8080808080808
3003 3960 

3264 3960 82.42424242424242
3265 3960 82.44949494949495
3266 3960 82.47474747474747
3267 3960 82.5
3268 3960 82.52525252525253
3269 3960 82.55050505050505
3270 3960 82.57575757575758
3271 3960 82.6010101010101
3272 3960 82.62626262626263
3273 3960 82.65151515151516
3274 3960 82.67676767676768
3275 3960 82.70202020202021
3276 3960 82.72727272727273
3277 3960 82.75252525252525
3278 3960 82.77777777777777
3279 3960 82.8030303030303
3280 3960 82.82828282828282
3281 3960 82.85353535353535
3282 3960 82.87878787878788
3283 3960 82.9040404040404
3284 3960 82.92929292929293
3285 3960 82.95454545454545
3286 3960 82.97979797979798
3287 3960 83.0050505050505
3288 3960 83.03030303030303
3289 3960 83.05555555555556
3290 3960 83.08080808080808
3291 3960 83.10606060606061
3292 3960 83.13131313131314
3293 3960 83.15656565656566
3294 3960 83.18181818181819
3295 3960 83.20707070707071
3296 3960 83.23232323232324
3297 3960 83.25757575757575
3298 3960 83.28282828282828
3299 3960 83.3080808080808
3300 3960 

3560 3960 89.8989898989899
3561 3960 89.92424242424242
3562 3960 89.94949494949495
3563 3960 89.97474747474747
3564 3960 90.0
3565 3960 90.02525252525253
3566 3960 90.05050505050505
3567 3960 90.07575757575758
3568 3960 90.1010101010101
3569 3960 90.12626262626263
3570 3960 90.15151515151516
3571 3960 90.17676767676768
3572 3960 90.20202020202021
3573 3960 90.22727272727273
3574 3960 90.25252525252525
3575 3960 90.27777777777777
3576 3960 90.3030303030303
3577 3960 90.32828282828282
3578 3960 90.35353535353535
3579 3960 90.37878787878788
3580 3960 90.4040404040404
3581 3960 90.42929292929293
3582 3960 90.45454545454545
3583 3960 90.47979797979798
3584 3960 90.5050505050505
3585 3960 90.53030303030303
3586 3960 90.55555555555556
3587 3960 90.58080808080808
3588 3960 90.60606060606061
3589 3960 90.63131313131314
3590 3960 90.65656565656566
3591 3960 90.68181818181819
3592 3960 90.70707070707071
3593 3960 90.73232323232324
3594 3960 90.75757575757575
3595 3960 90.78282828282828
3596 3960 

3856 3960 97.37373737373737
3857 3960 97.3989898989899
3858 3960 97.42424242424242
3859 3960 97.44949494949495
3860 3960 97.47474747474747
3861 3960 97.5
3862 3960 97.52525252525253
3863 3960 97.55050505050505
3864 3960 97.57575757575758
3865 3960 97.6010101010101
3866 3960 97.62626262626263
3867 3960 97.65151515151516
3868 3960 97.67676767676768
3869 3960 97.70202020202021
3870 3960 97.72727272727273
3871 3960 97.75252525252525
3872 3960 97.77777777777777
3873 3960 97.8030303030303
3874 3960 97.82828282828282
3875 3960 97.85353535353535
3876 3960 97.87878787878788
3877 3960 97.9040404040404
3878 3960 97.92929292929293
3879 3960 97.95454545454545
3880 3960 97.97979797979798
3881 3960 98.0050505050505
3882 3960 98.03030303030303
3883 3960 98.05555555555556
3884 3960 98.08080808080808
3885 3960 98.10606060606061
3886 3960 98.13131313131314
3887 3960 98.15656565656566
3888 3960 98.18181818181819
3889 3960 98.20707070707071
3890 3960 98.23232323232324
3891 3960 98.25757575757575
3892 3960 

In [31]:
# pickle.dump(pd.DataFrame(data_bertweet).set_index("id"), open("df_representation_v2.pickle", "wb"))

In [57]:
df_representation_v2 = pickle.load(open("df_representation_v2.pickle", "rb"))

In [58]:
df_representation_v2

,BERTweet_0,BERTweet_1,BERTweet_2,BERTweet_3,BERTweet_4,BERTweet_5,BERTweet_6,BERTweet_7,BERTweet_8,BERTweet_9,...,BERTweet_758,BERTweet_759,BERTweet_760,BERTweet_761,BERTweet_762,BERTweet_763,BERTweet_764,BERTweet_765,BERTweet_766,BERTweet_767
id,,,,,,,,,,,,,,,,,,,,,
10000,0.011935,-0.122075,0.029030,0.120095,0.093651,0.022820,0.006946,0.168774,0.282526,-0.009636,...,0.091612,0.013699,0.048113,0.180316,0.025295,-0.094723,0.094302,-0.039285,0.234481,-0.249347
10001,0.019873,-0.154840,0.128666,-0.013688,0.124787,-0.060633,0.029985,0.192368,0.358541,-0.000735,...,0.061139,0.057788,0.016066,0.114631,-0.075937,0.161129,0.088782,0.099523,0.195573,-0.094360
10002,0.089303,0.005199,0.056339,0.029113,0.114146,0.105520,0.041722,0.180096,0.075384,0.047837,...,0.058741,-0.221301,0.011673,0.277279,0.015945,0.010225,0.090561,0.177221,0.102671,-0.183728
10003,-0.174818,-0.169233,0.108006,0.089053,0.113685,-0.024810,0.045306,0.109823,0.190615,-0.004136,...,0.041260,0.074323,0.122148,0.129413,0.010133,-0.035607,0.095457,0.020883,0.222822,-0.049975
10004,0.016567,-0.013723,0.086389,-0.020837,0.122210,0.105154,0.083129,0.222264,0.327119,0.121932,...,0.152741,-0.128246,0.041543,0.219807,0.010124,0.019276,0.062688,0.044342,0.058079,-0.121317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40855,0.025625,-0.161436,0.085601,0.067647,0.092613,0.041396,0.085973,0.192643,0.252361,-0.007928,...,0.094263,0.087653,-0.054003,-0.014100,-0.015075,0.043808,0.258219,-0.013177,0.014293,0.006944
40856,-0.107447,0.002433,0.046188,0.105640,0.028643,0.141425,-0.002773,0.256521,0.372382,0.066727,...,0.090047,-0.136675,-0.018746,-0.072087,0.148523,-0.059273,0.200686,0.004628,0.182843,-0.168076
40857,-0.043438,0.011180,0.206375,0.071783,0.163780,-0.062001,-0.111886,0.200515,0.215893,0.117610,...,0.214575,-0.152968,-0.036713,0.054679,-0.097778,-0.037324,0.141307,0.059210,0.080930,-0.205299


In [85]:
sen = sentiments[3]

# indexs = df_train[df_train["sen"] == sen].index
# indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin(["low", "high"]))].index

# X = df_representation_v2.loc[indexsLH]#[rel_cols]
# y = df_train.loc[X.index]["int"]

# dic_label_count = y.value_counts().to_dict()
# min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
# max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
# index_label_1 = y[y==min_label].index
# oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1

# X_res, y_res = X.copy(), y.copy()
# for step in range(oversampling_steps):
#     new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
#     copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
#     copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
#     X_res = pd.concat([X_res, copied_sub_X], axis=0)
#     y_res = pd.concat([y_res, copied_sub_y], axis=0)

# X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
# selector = SelectKBest(chi2, k=X.shape[1])
# X_res_ = X_res - X_res.min()
# selector.fit(X_res_, y_res)
# scores_selector = {col: selector.scores_[i] if str(selector.scores_[i]) != "nan" else 0 for i, col in enumerate(X.columns.tolist())}
# ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda x: x[1], reverse=True)]
# print(ranked_cols)

indexs = df_train[df_train["sen"] == sen].index
scores_selector = {col: [] for col in df_representation_v2.columns.tolist()}
comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
for l1 in intensities:
    for l2 in intensities:
        if l1 < l2:
            indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

            X = df_representation_v2.loc[indexsLH]#[rel_cols]
            y = df_train.loc[X.index]["int"]

            dic_label_count = y.value_counts().to_dict()
            min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
            max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
            index_label_1 = y[y==min_label].index
            oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
            X_res, y_res = X.copy(), y.copy()

            for step in range(oversampling_steps):
                new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                X_res = pd.concat([X_res, copied_sub_X], axis=0)
                y_res = pd.concat([y_res, copied_sub_y], axis=0)

            X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
            selector = SelectKBest(chi2, k=X.shape[1])
            X_res_ = X_res - X_res.min()
            selector.fit(X_res_, y_res)
            for i, col in enumerate(X.columns.tolist()):
                scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]

f1_weight = []

for num_cols in range(1, len(ranked_cols)+1, 10):
    X = df_representation_v2.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(f1_weight[-1])

[1, 0.4266742580862261, 0.029000652362772134]
[11, 0.5117476531712855, 0.019080276048811858]
[21, 0.5691244234970079, 0.04990272605701973]
[31, 0.595363860354104, 0.06558170336976499]
[41, 0.6146930984747271, 0.0577397523311529]
[51, 0.6061797724110066, 0.051271561695346364]
[61, 0.6217853427952491, 0.05838768871150769]
[71, 0.6232044194188641, 0.05301549697307111]
[81, 0.6214658103794599, 0.05771478690843295]
[91, 0.624423847601709, 0.044536946833991856]
[101, 0.6262091752425574, 0.028067527409609536]
[111, 0.6185092326001929, 0.036389237557653296]
[121, 0.6235964982309857, 0.03725058865965338]
[131, 0.6284264570252145, 0.03253518489236862]
[141, 0.6195900873245852, 0.0296541695143281]
[151, 0.617291930346149, 0.03715468479851581]
[161, 0.6260251365223966, 0.03725010209419756]
[171, 0.6226468402527421, 0.045070654836315664]
[181, 0.608469030245583, 0.053909784873922874]
[191, 0.6115024692531071, 0.053040255651115885]
[201, 0.6024820168684965, 0.053295301481507115]
[211, 0.610737709918

In [86]:
best_f1 = sorted(f1_weight, key=lambda x: x[1], reverse=True)[0][0]
best_f1

231

In [87]:
fine_f1_weight = []

for num_cols in range(best_f1-10, best_f1+10, 1):
    X = df_representation_v2.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    fine_f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(fine_f1_weight[-1])

[221, 0.6161255963718444, 0.03959949641766339]
[222, 0.6186198803002875, 0.036407568095489805]
[223, 0.6138463310886813, 0.043263347293339326]
[224, 0.6143611228959699, 0.03579301381324165]
[225, 0.6167498721827913, 0.03468169668907546]
[226, 0.622951465955635, 0.026664902471686684]
[227, 0.6229226948672073, 0.031095527637208215]
[228, 0.6239504164751969, 0.03321562044884642]
[229, 0.6255463082581119, 0.03356620439571124]
[230, 0.6327171768763874, 0.03932064083309087]
[231, 0.6339039135605046, 0.04073463436585979]
[232, 0.6327304831336775, 0.039429697311889514]
[233, 0.6327725983950642, 0.03841928757799965]
[234, 0.6269059225585256, 0.03663145007951668]
[235, 0.6259100504572533, 0.04273009908116228]
[236, 0.6268939376593157, 0.04169930241562819]
[237, 0.6262862090433599, 0.043669043514568004]
[238, 0.6265687269378318, 0.04438889611066611]
[239, 0.6304088107130051, 0.03911324370210424]
[240, 0.6292573854649839, 0.03802969201016805]


In [88]:
fine_best_f1 = sorted(fine_f1_weight, key=lambda x: x[1], reverse=True)[0][0]
fine_best_f1

231

In [84]:
pickle.dump(ranked_cols[:fine_best_f1+1], open(f"cols_selected_{sen}_v2.pickle", "wb"))

In [75]:
df_representation_v3 = df_representation
df_representation_v3 = pd.concat([df_representation_v3, df_representation_v2], axis=1)
df_representation_v3

,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,retro<&>num_tokens_upper,retro<&>prop_vowels,retro<&>len_max_rep_char,retro<&>max_char_fre_per_token(o),retro<&>max_char_fre_per_token(s),...,BERTweet_758,BERTweet_759,BERTweet_760,BERTweet_761,BERTweet_762,BERTweet_763,BERTweet_764,BERTweet_765,BERTweet_766,BERTweet_767
id,,,,,,,,,,,,,,,,,,,,,
10000,18,96,0,18,3,1,0.253165,2,2,1,...,0.091612,0.013699,0.048113,0.180316,0.025295,-0.094723,0.094302,-0.039285,0.234481,-0.249347
10001,23,119,0,25,7,2,0.329897,2,2,2,...,0.061139,0.057788,0.016066,0.114631,-0.075937,0.161129,0.088782,0.099523,0.195573,-0.094360
10002,19,108,0,19,2,2,0.333333,2,3,2,...,0.058741,-0.221301,0.011673,0.277279,0.015945,0.010225,0.090561,0.177221,0.102671,-0.183728
10003,24,134,0,23,1,1,0.315315,2,1,2,...,0.041260,0.074323,0.122148,0.129413,0.010133,-0.035607,0.095457,0.020883,0.222822,-0.049975
10004,24,125,0,25,3,0,0.352941,2,1,0,...,0.152741,-0.128246,0.041543,0.219807,0.010124,0.019276,0.062688,0.044342,0.058079,-0.121317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40855,24,112,0,23,2,1,0.325843,2,2,2,...,0.094263,0.087653,-0.054003,-0.014100,-0.015075,0.043808,0.258219,-0.013177,0.014293,0.006944
40856,20,121,0,21,1,0,0.352941,2,1,3,...,0.090047,-0.136675,-0.018746,-0.072087,0.148523,-0.059273,0.200686,0.004628,0.182843,-0.168076
40857,23,139,0,23,2,1,0.316239,2,2,4,...,0.214575,-0.152968,-0.036713,0.054679,-0.097778,-0.037324,0.141307,0.059210,0.080930,-0.205299


In [35]:
# pickle.dump(df_representation_v3, open("df_representation_v3.pickle", "wb"))

In [109]:
sen = sentiments[0]

# indexs = df_train[df_train["sen"] == sen].index
# indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin(["low", "high"]))].index

# X = df_representation_v3.loc[indexsLH]#[rel_cols]
# y = df_train.loc[X.index]["int"]

# dic_label_count = y.value_counts().to_dict()
# min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
# max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
# index_label_1 = y[y==min_label].index
# oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1

# X_res, y_res = X.copy(), y.copy()
# for step in range(oversampling_steps):
#     new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
#     copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
#     copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
#     X_res = pd.concat([X_res, copied_sub_X], axis=0)
#     y_res = pd.concat([y_res, copied_sub_y], axis=0)

# X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
# selector = SelectKBest(chi2, k=X.shape[1])
# X_res_ = X_res - X_res.min()
# selector.fit(X_res_, y_res)
# scores_selector = {col: selector.scores_[i] if str(selector.scores_[i]) != "nan" else 0 for i, col in enumerate(X.columns.tolist())}
# ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda x: x[1], reverse=True)]
# print(ranked_cols)

indexs = df_train[df_train["sen"] == sen].index
scores_selector = {col: [] for col in df_representation_v3.columns.tolist()}
comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
for l1 in intensities:
    for l2 in intensities:
        if l1 < l2:
            indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

            X = df_representation_v3.loc[indexsLH]#[rel_cols]
            y = df_train.loc[X.index]["int"]

            dic_label_count = y.value_counts().to_dict()
            min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
            max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
            index_label_1 = y[y==min_label].index
            oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
            X_res, y_res = X.copy(), y.copy()

            for step in range(oversampling_steps):
                new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                X_res = pd.concat([X_res, copied_sub_X], axis=0)
                y_res = pd.concat([y_res, copied_sub_y], axis=0)

            X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
            selector = SelectKBest(chi2, k=X.shape[1])
            X_res_ = X_res - X_res.min()
            selector.fit(X_res_, y_res)
            for i, col in enumerate(X.columns.tolist()):
                scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]

f1_weight = []

for num_cols in range(1, len(ranked_cols)+1, 10):
    X = df_representation_v3.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(f1_weight[-1])

[1, 0.10424886540056362, 0.019309878014991615]
[11, 0.6411688350633156, 0.029228674134960844]
[21, 0.6609119066959994, 0.02203836650268399]
[31, 0.6588559737273805, 0.012008818616047677]
[41, 0.66215257900806, 0.004897267596503262]
[51, 0.6819780207011394, 0.018291375701466977]
[61, 0.6891776153090093, 0.018405652049383794]
[71, 0.6948351734809612, 0.026925106785193546]
[81, 0.698515744491627, 0.030620007364956892]
[91, 0.6995012251275626, 0.029302400431879483]
[101, 0.7001325185391131, 0.029258873024053687]
[111, 0.7115093975305629, 0.02715503116164831]
[121, 0.710112986921863, 0.02801785329972967]
[131, 0.7000463757225261, 0.03418087393899213]
[141, 0.7062307214260791, 0.03812506665278833]
[151, 0.7079706264171842, 0.03664070574426643]
[161, 0.7036688956016317, 0.038892980889282364]
[171, 0.7010408157744935, 0.04568406926917083]
[181, 0.6933352645072454, 0.032473129834692624]
[191, 0.6942896187745584, 0.04011906590259415]
[201, 0.6928814435108482, 0.043088935779638876]
[211, 0.686060

KeyboardInterrupt: 

In [110]:
best_f1 = sorted(f1_weight, key=lambda x: x[1], reverse=True)[0][0]
best_f1

721

In [111]:
fine_f1_weight = []

for num_cols in range(best_f1-10, best_f1+10, 1):
    X = df_representation_v3.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    fine_f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(fine_f1_weight[-1])

[711, 0.7119090707998, 0.027855017461954637]
[712, 0.7136188549235528, 0.025519407164058552]
[713, 0.7136188549235528, 0.025519407164058552]
[714, 0.7136188549235528, 0.025519407164058552]
[715, 0.7147670371058765, 0.02471609889494386]
[716, 0.7152799717726143, 0.02237807015865809]
[717, 0.7152205637645912, 0.02236798949000108]
[718, 0.7152205637645912, 0.02236798949000108]
[719, 0.7152205637645912, 0.02236798949000108]
[720, 0.7139066277577931, 0.023421575299425103]
[721, 0.7139066277577931, 0.023421575299425103]
[722, 0.7130921410103506, 0.02330435003286564]
[723, 0.7130204092882748, 0.02329945517365635]
[724, 0.7130204092882748, 0.02329945517365635]
[725, 0.7130204092882748, 0.02329945517365635]
[726, 0.7138116315671187, 0.022539456984139863]
[727, 0.7138116315671187, 0.022539456984139863]
[728, 0.7138116315671187, 0.022539456984139863]
[729, 0.7138116315671187, 0.022539456984139863]
[730, 0.7138116315671187, 0.022539456984139863]


In [112]:
fine_best_f1 = sorted(fine_f1_weight, key=lambda x: x[1], reverse=True)[0][0]
fine_best_f1

716

In [113]:
pickle.dump(ranked_cols[:fine_best_f1+1], open(f"cols_selected_{sen}_v3.pickle", "wb"))

In [218]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('opinion_lexicon')
# nltk.download("sentiwordnet")
# nltk.download('omw-1.4')

from nltk.corpus import opinion_lexicon
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [219]:
def get_sentiwordnet_sent(tweet):
    def penn_to_wn(tag):
        if tag.startswith('J'):
            return wn.ADJ
        elif tag.startswith('N'):
            return wn.NOUN
        elif tag.startswith('R'):
            return wn.ADV
        elif tag.startswith('V'):
            return wn.VERB
        return None

    def get_sentiment(word,tag):
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            return [0,0,0]

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            return [0,0,0]

        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            return [0,0,0]

        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())

        return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    words_data = tweet.lower().strip().split()

    pos_val = nltk.pos_tag(words_data)
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    return dict(zip("+ - o".split(), np.sum(senti_val, axis=0)))

In [220]:
print(f"Cantidad de palabras positivas: {len(opinion_lexicon.positive())}")
print(f"Cantidad de palabras negativas: {len(opinion_lexicon.negative())}")

Cantidad de palabras positivas: 2006
Cantidad de palabras negativas: 4783


In [221]:
def get_lexicon_attrib(tweet):
    o = {}
    o["lexicon<&>LiuHu<&>+"] = sum(int(t.lower() in opinion_lexicon.positive()) for t in tweet.split())
    o["lexicon<&>LiuHu<&>-"] = sum(int(t.lower() in opinion_lexicon.negative()) for t in tweet.split())
    o_sentiwordnet = get_sentiwordnet_sent(tweet)
    o["lexicon<&>sentiwordnet<&>+"] = o_sentiwordnet["+"]
    o["lexicon<&>sentiwordnet<&>-"] = o_sentiwordnet["-"]
    o["lexicon<&>sentiwordnet<&>o"] = o_sentiwordnet["o"]
    return o

In [222]:
get_lexicon_attrib(tweet)

{'lexicon<&>LiuHu<&>+': 0,
 'lexicon<&>LiuHu<&>-': 0,
 'lexicon<&>sentiwordnet<&>+': 0.75,
 'lexicon<&>sentiwordnet<&>-': 0.25,
 'lexicon<&>sentiwordnet<&>o': 6.0}

In [41]:
data_lexicon_attributes = []
times = []
for k, ix in enumerate(df_train.index):
    start_time = time.time()
    tweet = df_train.loc[ix]["text"]
    o = {"id": ix}
    o = {**o, **get_lexicon_attrib(tweet)}
    data_lexicon_attributes.append(o)
    dt = time.time()-start_time
    times.append(dt)
    print(f"""
    row: {k+1}, total_rows: {df_train.shape[0]}
    progress: {np.round(100*(k+1)/df_train.shape[0], 3)}%
    wait time: {datetime.timedelta(seconds = np.median(times)*(df_train.shape[0]-k-1))}s
    """)


    row: 1, total_rows: 3960
    progress: 0.025%
    wait time: 0:14:39.696078s
    

    row: 2, total_rows: 3960
    progress: 0.051%
    wait time: 0:16:38.320869s
    

    row: 3, total_rows: 3960
    progress: 0.076%
    wait time: 0:15:34.698491s
    

    row: 4, total_rows: 3960
    progress: 0.101%
    wait time: 0:17:05.532814s
    

    row: 5, total_rows: 3960
    progress: 0.126%
    wait time: 0:18:36.321094s
    

    row: 6, total_rows: 3960
    progress: 0.152%
    wait time: 0:17:05.014344s
    

    row: 7, total_rows: 3960
    progress: 0.177%
    wait time: 0:18:35.756583s
    

    row: 8, total_rows: 3960
    progress: 0.202%
    wait time: 0:17:55.920444s
    

    row: 9, total_rows: 3960
    progress: 0.227%
    wait time: 0:17:16.104323s
    

    row: 10, total_rows: 3960
    progress: 0.253%
    wait time: 0:16:24.443539s
    

    row: 11, total_rows: 3960
    progress: 0.278%
    wait time: 0:15:32.808780s
    

    row: 12, total_rows: 3960
    progre


    row: 96, total_rows: 3960
    progress: 2.424%
    wait time: 0:13:47.645845s
    

    row: 97, total_rows: 3960
    progress: 2.449%
    wait time: 0:13:51.298055s
    

    row: 98, total_rows: 3960
    progress: 2.475%
    wait time: 0:14:04.612692s
    

    row: 99, total_rows: 3960
    progress: 2.5%
    wait time: 0:14:17.920323s
    

    row: 100, total_rows: 3960
    progress: 2.525%
    wait time: 0:14:04.175296s
    

    row: 101, total_rows: 3960
    progress: 2.551%
    wait time: 0:13:50.437276s
    

    row: 102, total_rows: 3960
    progress: 2.576%
    wait time: 0:13:46.360681s
    

    row: 103, total_rows: 3960
    progress: 2.601%
    wait time: 0:13:50.006886s
    

    row: 104, total_rows: 3960
    progress: 2.626%
    wait time: 0:13:45.932293s
    

    row: 105, total_rows: 3960
    progress: 2.652%
    wait time: 0:13:41.859702s
    

    row: 106, total_rows: 3960
    progress: 2.677%
    wait time: 0:13:45.503905s
    

    row: 107, total_rows: 


    row: 189, total_rows: 3960
    progress: 4.773%
    wait time: 0:12:46.206447s
    

    row: 190, total_rows: 3960
    progress: 4.798%
    wait time: 0:12:44.118849s
    

    row: 191, total_rows: 3960
    progress: 4.823%
    wait time: 0:12:45.800079s
    

    row: 192, total_rows: 3960
    progress: 4.848%
    wait time: 0:12:49.367766s
    

    row: 193, total_rows: 3960
    progress: 4.874%
    wait time: 0:12:45.393711s
    

    row: 194, total_rows: 3960
    progress: 4.899%
    wait time: 0:12:43.308113s
    

    row: 195, total_rows: 3960
    progress: 4.924%
    wait time: 0:12:41.223514s
    

    row: 196, total_rows: 3960
    progress: 4.949%
    wait time: 0:12:42.902745s
    

    row: 197, total_rows: 3960
    progress: 4.975%
    wait time: 0:12:44.580976s
    

    row: 198, total_rows: 3960
    progress: 5.0%
    wait time: 0:12:48.142658s
    

    row: 199, total_rows: 3960
    progress: 5.025%
    wait time: 0:12:51.702340s
    

    row: 200, total_ro


    row: 283, total_rows: 3960
    progress: 7.146%
    wait time: 0:12:49.191232s
    

    row: 284, total_rows: 3960
    progress: 7.172%
    wait time: 0:12:50.822100s
    

    row: 285, total_rows: 3960
    progress: 7.197%
    wait time: 0:12:48.772852s
    

    row: 286, total_rows: 3960
    progress: 7.222%
    wait time: 0:12:46.725044s
    

    row: 287, total_rows: 3960
    progress: 7.247%
    wait time: 0:12:48.354473s
    

    row: 288, total_rows: 3960
    progress: 7.273%
    wait time: 0:12:46.307665s
    

    row: 289, total_rows: 3960
    progress: 7.298%
    wait time: 0:12:47.936093s
    

    row: 290, total_rows: 3960
    progress: 7.323%
    wait time: 0:12:45.890286s
    

    row: 291, total_rows: 3960
    progress: 7.348%
    wait time: 0:12:47.517713s
    

    row: 292, total_rows: 3960
    progress: 7.374%
    wait time: 0:12:45.472907s
    

    row: 293, total_rows: 3960
    progress: 7.399%
    wait time: 0:12:47.099333s
    

    row: 294, total_


    row: 377, total_rows: 3960
    progress: 9.52%
    wait time: 0:12:00.840687s
    

    row: 378, total_rows: 3960
    progress: 9.545%
    wait time: 0:12:02.431654s
    

    row: 379, total_rows: 3960
    progress: 9.571%
    wait time: 0:12:00.438319s
    

    row: 380, total_rows: 3960
    progress: 9.596%
    wait time: 0:12:00.236282s
    

    row: 381, total_rows: 3960
    progress: 9.621%
    wait time: 0:12:00.035952s
    

    row: 382, total_rows: 3960
    progress: 9.646%
    wait time: 0:11:59.833916s
    

    row: 383, total_rows: 3960
    progress: 9.672%
    wait time: 0:11:59.631880s
    

    row: 384, total_rows: 3960
    progress: 9.697%
    wait time: 0:11:59.431549s
    

    row: 385, total_rows: 3960
    progress: 9.722%
    wait time: 0:11:59.229513s
    

    row: 386, total_rows: 3960
    progress: 9.747%
    wait time: 0:11:59.028330s
    

    row: 387, total_rows: 3960
    progress: 9.773%
    wait time: 0:11:58.827147s
    

    row: 388, total_r

...


    row: 3587, total_rows: 3960
    progress: 90.581%
    wait time: 0:01:13.547708s
    

    row: 3588, total_rows: 3960
    progress: 90.606%
    wait time: 0:01:13.350529s
    

    row: 3589, total_rows: 3960
    progress: 90.631%
    wait time: 0:01:13.153350s
    

    row: 3590, total_rows: 3960
    progress: 90.657%
    wait time: 0:01:12.956215s
    

    row: 3591, total_rows: 3960
    progress: 90.682%
    wait time: 0:01:12.759080s
    

    row: 3592, total_rows: 3960
    progress: 90.707%
    wait time: 0:01:12.561857s
    

    row: 3593, total_rows: 3960
    progress: 90.732%
    wait time: 0:01:12.364722s
    

    row: 3594, total_rows: 3960
    progress: 90.758%
    wait time: 0:01:12.167543s
    

    row: 3595, total_rows: 3960
    progress: 90.783%
    wait time: 0:01:11.970364s
    

    row: 3596, total_rows: 3960
    progress: 90.808%
    wait time: 0:01:11.773185s
    

    row: 3597, total_rows: 3960
    progress: 90.833%
    wait time: 0:01:11.576006s
    


    row: 3678, total_rows: 3960
    progress: 92.879%
    wait time: 0:00:55.745456s
    

    row: 3679, total_rows: 3960
    progress: 92.904%
    wait time: 0:00:55.407522s
    

    row: 3680, total_rows: 3960
    progress: 92.929%
    wait time: 0:00:55.350099s
    

    row: 3681, total_rows: 3960
    progress: 92.955%
    wait time: 0:00:55.291677s
    

    row: 3682, total_rows: 3960
    progress: 92.98%
    wait time: 0:00:54.954741s
    

    row: 3683, total_rows: 3960
    progress: 93.005%
    wait time: 0:00:54.618803s
    

    row: 3684, total_rows: 3960
    progress: 93.03%
    wait time: 0:00:54.559383s
    

    row: 3685, total_rows: 3960
    progress: 93.056%
    wait time: 0:00:54.498965s
    

    row: 3686, total_rows: 3960
    progress: 93.081%
    wait time: 0:00:54.164025s
    

    row: 3687, total_rows: 3960
    progress: 93.106%
    wait time: 0:00:54.102608s
    

    row: 3688, total_rows: 3960
    progress: 93.131%
    wait time: 0:00:53.904495s
    




    row: 3769, total_rows: 3960
    progress: 95.177%
    wait time: 0:00:37.852008s
    

    row: 3770, total_rows: 3960
    progress: 95.202%
    wait time: 0:00:37.653875s
    

    row: 3771, total_rows: 3960
    progress: 95.227%
    wait time: 0:00:37.455652s
    

    row: 3772, total_rows: 3960
    progress: 95.253%
    wait time: 0:00:37.163638s
    

    row: 3773, total_rows: 3960
    progress: 95.278%
    wait time: 0:00:37.059296s
    

    row: 3774, total_rows: 3960
    progress: 95.303%
    wait time: 0:00:36.768280s
    

    row: 3775, total_rows: 3960
    progress: 95.328%
    wait time: 0:00:36.478262s
    

    row: 3776, total_rows: 3960
    progress: 95.354%
    wait time: 0:00:36.281082s
    

    row: 3777, total_rows: 3960
    progress: 95.379%
    wait time: 0:00:36.083902s
    

    row: 3778, total_rows: 3960
    progress: 95.404%
    wait time: 0:00:35.977564s
    

    row: 3779, total_rows: 3960
    progress: 95.429%
    wait time: 0:00:35.870228s
    


    row: 3860, total_rows: 3960
    progress: 97.475%
    wait time: 0:00:19.817913s
    

    row: 3861, total_rows: 3960
    progress: 97.5%
    wait time: 0:00:19.619745s
    

    row: 3862, total_rows: 3960
    progress: 97.525%
    wait time: 0:00:19.421554s
    

    row: 3863, total_rows: 3960
    progress: 97.551%
    wait time: 0:00:19.223387s
    

    row: 3864, total_rows: 3960
    progress: 97.576%
    wait time: 0:00:19.025196s
    

    row: 3865, total_rows: 3960
    progress: 97.601%
    wait time: 0:00:18.827028s
    

    row: 3866, total_rows: 3960
    progress: 97.626%
    wait time: 0:00:18.628849s
    

    row: 3867, total_rows: 3960
    progress: 97.652%
    wait time: 0:00:18.430670s
    

    row: 3868, total_rows: 3960
    progress: 97.677%
    wait time: 0:00:18.232491s
    

    row: 3869, total_rows: 3960
    progress: 97.702%
    wait time: 0:00:18.034311s
    

    row: 3870, total_rows: 3960
    progress: 97.727%
    wait time: 0:00:17.836132s
    




    row: 3951, total_rows: 3960
    progress: 99.773%
    wait time: 0:00:01.783615s
    

    row: 3952, total_rows: 3960
    progress: 99.798%
    wait time: 0:00:01.585436s
    

    row: 3953, total_rows: 3960
    progress: 99.823%
    wait time: 0:00:01.387256s
    

    row: 3954, total_rows: 3960
    progress: 99.848%
    wait time: 0:00:01.189077s
    

    row: 3955, total_rows: 3960
    progress: 99.874%
    wait time: 0:00:00.990897s
    

    row: 3956, total_rows: 3960
    progress: 99.899%
    wait time: 0:00:00.792718s
    

    row: 3957, total_rows: 3960
    progress: 99.924%
    wait time: 0:00:00.594538s
    

    row: 3958, total_rows: 3960
    progress: 99.949%
    wait time: 0:00:00.396359s
    

    row: 3959, total_rows: 3960
    progress: 99.975%
    wait time: 0:00:00.198179s
    

    row: 3960, total_rows: 3960
    progress: 100.0%
    wait time: 0:00:00s
    


In [116]:
df_representation_v4 = pd.DataFrame(data_lexicon_attributes).set_index("id")
df_representation_v4 = pd.concat([df_representation_v4, df_representation_v3], axis=1)
df_representation_v4

NameError: name 'data_lexicon_attributes' is not defined

In [118]:
# pickle.dump(df_representation_v4, open("df_representation_v4.pickle", "wb"))
# df_representation_v4 = pickle.load(open("df_representation_v4.pickle", "rb"))
# df_representation_v4 = df_representation_v4[[c for c in df_representation_v4.columns if "lexicon<&>" in c]]
# df_representation_v4 = pd.concat([df_representation_v4, df_representation_v3], axis=1)
# df_representation_v4

,lexicon<&>LiuHu<&>+,lexicon<&>LiuHu<&>-,lexicon<&>sentiwordnet<&>+,lexicon<&>sentiwordnet<&>-,lexicon<&>sentiwordnet<&>o,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,...,BERTweet_758,BERTweet_759,BERTweet_760,BERTweet_761,BERTweet_762,BERTweet_763,BERTweet_764,BERTweet_765,BERTweet_766,BERTweet_767
id,,,,,,,,,,,,,,,,,,,,,
10000,0,1,0.000,0.000,1.000,18,96,0,18,3,...,0.091612,0.013699,0.048113,0.180316,0.025295,-0.094723,0.094302,-0.039285,0.234481,-0.249347
10001,0,0,0.000,0.250,7.750,23,119,0,25,7,...,0.061139,0.057788,0.016066,0.114631,-0.075937,0.161129,0.088782,0.099523,0.195573,-0.094360
10002,0,0,0.000,0.625,6.375,19,108,0,19,2,...,0.058741,-0.221301,0.011673,0.277279,0.015945,0.010225,0.090561,0.177221,0.102671,-0.183728
10003,0,1,1.125,0.250,5.625,24,134,0,23,1,...,0.041260,0.074323,0.122148,0.129413,0.010133,-0.035607,0.095457,0.020883,0.222822,-0.049975
10004,0,0,0.000,0.250,4.750,24,125,0,25,3,...,0.152741,-0.128246,0.041543,0.219807,0.010124,0.019276,0.062688,0.044342,0.058079,-0.121317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40855,0,2,0.375,0.250,7.375,24,112,0,23,2,...,0.094263,0.087653,-0.054003,-0.014100,-0.015075,0.043808,0.258219,-0.013177,0.014293,0.006944
40856,1,0,1.500,0.500,3.000,20,121,0,21,1,...,0.090047,-0.136675,-0.018746,-0.072087,0.148523,-0.059273,0.200686,0.004628,0.182843,-0.168076
40857,0,0,0.750,0.000,2.250,23,139,0,23,2,...,0.214575,-0.152968,-0.036713,0.054679,-0.097778,-0.037324,0.141307,0.059210,0.080930,-0.205299


In [178]:
# pickle.dump(df_representation_v4, open("df_representation_v4.pickle", "wb"))

In [363]:
sen = sentiments[0]

# indexs = df_train[df_train["sen"] == sen].index
# indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin(["low", "high"]))].index

# X = df_representation_v4.loc[indexsLH]#[rel_cols]
# y = df_train.loc[X.index]["int"]

# dic_label_count = y.value_counts().to_dict()
# min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
# max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
# index_label_1 = y[y==min_label].index
# oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1

# X_res, y_res = X.copy(), y.copy()
# for step in range(oversampling_steps):
#     new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
#     copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
#     copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
#     X_res = pd.concat([X_res, copied_sub_X], axis=0)
#     y_res = pd.concat([y_res, copied_sub_y], axis=0)

# X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
# selector = SelectKBest(chi2, k=X.shape[1])
# X_res_ = X_res - X_res.min()
# selector.fit(X_res_, y_res)
# scores_selector = {col: selector.scores_[i] if str(selector.scores_[i]) != "nan" else 0 for i, col in enumerate(X.columns.tolist())}
# ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda x: x[1], reverse=True)]
# print(ranked_cols)

indexs = df_train[df_train["sen"] == sen].index
scores_selector = {col: [] for col in df_representation_v4.columns.tolist()}
comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
for l1 in intensities:
    for l2 in intensities:
        if l1 < l2:
            indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

            X = df_representation_v4.loc[indexsLH]#[rel_cols]
            y = df_train.loc[X.index]["int"]

            dic_label_count = y.value_counts().to_dict()
            min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
            max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
            index_label_1 = y[y==min_label].index
            oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
            X_res, y_res = X.copy(), y.copy()

            for step in range(oversampling_steps):
                new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                X_res = pd.concat([X_res, copied_sub_X], axis=0)
                y_res = pd.concat([y_res, copied_sub_y], axis=0)

            X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
            selector = SelectKBest(chi2, k=X.shape[1])
            X_res_ = X_res - X_res.min()
            selector.fit(X_res_, y_res)
            for i, col in enumerate(X.columns.tolist()):
                scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]


f1_weight = []

for num_cols in range(1, len(ranked_cols)+1, 10):
    X = df_representation_v4.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(f1_weight[-1])

[10, 0.269252565656121, 0.02290469784861938]
0.0
[20, 0.2860016373313831, 0.024583514937789906]
0.0
[30, 0.33076135719898636, 0.017229772075774856]
0.0
[40, 0.3479687056866973, 0.030236108736200792]
0.0
[50, 0.3771521982927939, 0.0746230044378462]
0.0
[60, 0.3996847995111568, 0.04144134271314507]
0.0
[70, 0.4267900688789351, 0.05983731131658159]
0.0
[80, 0.45470957220675745, 0.08410853080338244]
0.0
[90, 0.5011042060899384, 0.09963893271964369]
0.0
[100, 0.5083544506129323, 0.10320718275703097]
0.0
[110, 0.5062030619499056, 0.1024069822254258]
0.0
[120, 0.5049054223742787, 0.1036636277641506]
0.0
[130, 0.5454862629398902, 0.10287608842056734]
0.0
[140, 0.5466147734493683, 0.1039515551946309]
0.0
[150, 0.5458826594113458, 0.10346433981180436]
0.0
[160, 0.5557256350934482, 0.10366834115383063]
0.0
[170, 0.5580013550582306, 0.10446034550950009]
0.0
[180, 0.5592190913496725, 0.11125537541340726]
0.0
[190, 0.6110727233059613, 0.03166581842338372]
0.0
[200, 0.6129803083756296, 0.031134681135

379

In [365]:
best_f1 = sorted(f1_weight, key=lambda x: x[1], reverse=True)[0][0]
best_f1

380

In [175]:
fine_f1_weight = []

for num_cols in range(best_f1-10, best_f1+10, 1):
    X = df_representation_v4.loc[indexs][ranked_cols[:num_cols+1]]
    y = df_train.loc[X.index]["int"]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
    # clf.fit(X_train, y_train)
    cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
    test_score = cv_results["test_score"]
    fine_f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
    print(fine_f1_weight[-1])

[711, 0.7135064054634677, 0.02843300232297205]
[712, 0.7135064054634677, 0.02843300232297205]
[713, 0.7152161895872203, 0.02604441533996918]
[714, 0.7152161895872203, 0.02604441533996918]
[715, 0.7152161895872203, 0.02604441533996918]
[716, 0.7154814838404342, 0.02646863771210781]
[717, 0.7175227985191704, 0.02487539412245578]
[718, 0.7148818500130721, 0.02404939261838799]
[719, 0.7148818500130721, 0.02404939261838799]
[720, 0.7148818500130721, 0.02404939261838799]
[721, 0.7148818500130721, 0.02404939261838799]
[722, 0.7148818500130721, 0.02404939261838799]
[723, 0.7140130503602529, 0.023388058654153963]
[724, 0.7139676914418421, 0.023356567742729654]
[725, 0.7124241789966591, 0.022477710945360958]
[726, 0.7133086362027828, 0.021444504015364445]
[727, 0.7140998584816268, 0.0206051731151041]
[728, 0.7140998584816268, 0.0206051731151041]
[729, 0.7140998584816268, 0.0206051731151041]
[730, 0.7140998584816268, 0.0206051731151041]


In [176]:
fine_best_f1 = sorted(fine_f1_weight, key=lambda x: x[1], reverse=True)[0][0]
fine_best_f1

717

In [368]:
def feature_selection_chi2(sen, df_rep, df_train):
    indexs = df_train[df_train["sen"] == sen].index
    scores_selector = {col: [] for col in df_rep.columns.tolist()}
    comp_l = [(l1, l2) for l1 in intensities for l2 in intensities if l1 < l2]
    for l1 in intensities:
        for l2 in intensities:
            if l1 < l2:
                indexsLH = df_train.loc[indexs][(df_train.loc[indexs]["int"].isin([l1, l2]))].index

                X = df_rep.loc[indexsLH]#[rel_cols]
                y = df_train.loc[X.index]["int"]

                dic_label_count = y.value_counts().to_dict()
                min_label = min(dic_label_count.items(), key=lambda x: x[1])[0]
                max_label = max(dic_label_count.items(), key=lambda x: x[1])[0]
                index_label_1 = y[y==min_label].index
                oversampling_steps = int(dic_label_count[max_label] / dic_label_count[min_label]) - 1
                X_res, y_res = X.copy(), y.copy()

                for step in range(oversampling_steps):
                    new_indexs = [f"{ix}+{step + 1}" for ix in index_label_1]
                    copied_sub_X = pd.DataFrame(X.loc[index_label_1].values, columns=X.columns, index=new_indexs)
                    copied_sub_y = pd.Series(y.loc[index_label_1].values, index=new_indexs)
                    X_res = pd.concat([X_res, copied_sub_X], axis=0)
                    y_res = pd.concat([y_res, copied_sub_y], axis=0)

                X_res = pd.DataFrame(StandardScaler().fit_transform(X_res), columns=X_res.columns, index=X_res.index)
                selector = SelectKBest(chi2, k=X.shape[1])
                X_res_ = X_res - X_res.min()
                selector.fit(X_res_, y_res)
                for i, col in enumerate(X.columns.tolist()):
                    scores_selector[col] += [selector.scores_[i]] if str(selector.scores_[i]) != "nan" else [0]

    ranked_cols = [x[0] for x in sorted(scores_selector.items(), key=lambda y: max(y[1]), reverse=True)]


    f1_weight = []
    b = []
    p = []
    pp = []
    ppp = []
    for num_cols in range(10, len(ranked_cols)+1, 10):
        X = df_rep.loc[indexs][ranked_cols[:num_cols+1]]
        y = df_train.loc[X.index]["int"]

        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
        # clf.fit(X_train, y_train)
        cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
        test_score = cv_results["test_score"]
        f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
        print(f1_weight[-1])
        b.append(f1_weight[-1][1])
        p.append(np.mean(b[max(0, len(b)-15-1):]))
        pp.append(p[-1]-p[-min(2, len(p))])
        ppp.append(np.mean(pp[max(0, len(pp)-30-1):]))
        pppc = np.mean([int(x<0) for x in ppp[max(0, len(ppp)-5-1):]])
        print(pppc)
        if int(pppc) == 1:
            break

    best_f1 = sorted(f1_weight, key=lambda x: x[1], reverse=True)[0][0]

    fine_f1_weight = []

    for num_cols in range(best_f1-10, best_f1+10, 1):
        X = df_rep.loc[indexs][ranked_cols[:num_cols+1]]
        y = df_train.loc[X.index]["int"]

        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", gamma='auto', class_weight="balanced"))
        # clf.fit(X_train, y_train)
        cv_results = cross_validate(clf, X, y, cv=5, scoring="f1_weighted")
        test_score = cv_results["test_score"]
        fine_f1_weight.append([num_cols, np.mean(test_score), np.std(test_score)])
        print(fine_f1_weight[-1])

    fine_best_f1 = sorted(fine_f1_weight, key=lambda x: x[1], reverse=True)[0]
    return fine_best_f1, ranked_cols

In [373]:
sen = sentiments[0]
print(sen)
df_rep = df_representation_v2

fine_best_f1, ranked_cols = feature_selection_chi2(sen, df_rep, df_train)
fine_best_f1

anger
[10, 0.541673907368766, 0.04808507760750921]
0.0
[20, 0.588934438579573, 0.04710427073113271]
0.0
[30, 0.6381617441626649, 0.06651217881200026]
0.0
[40, 0.6259487297652131, 0.0562376857919119]
0.0
[50, 0.6377911789844732, 0.06588976328824146]
0.0
[60, 0.6411370972438577, 0.05677144065549385]
0.0
[70, 0.6561020754926861, 0.05265189778103993]
0.0
[80, 0.6671945722948932, 0.055916890873644175]
0.0
[90, 0.6715423991240267, 0.05894946826338468]
0.0
[100, 0.6807897823719811, 0.04921030962252635]
0.0
[110, 0.671135241781607, 0.056163696758723035]
0.0
[120, 0.6698781496818711, 0.05133433819243292]
0.0
[130, 0.6601968601744916, 0.05570385812455512]
0.0
[140, 0.6671737921947196, 0.0634762171414373]
0.0
[150, 0.680152519080467, 0.055171714658180306]
0.0
[160, 0.6851660275345197, 0.05844286181524831]
0.0
[170, 0.6770272643824486, 0.06210400845933777]
0.0
[180, 0.6870741365381422, 0.0671224156596285]
0.0
[190, 0.6810553938179604, 0.05856752538506118]
0.0
[200, 0.6845487497093083, 0.0659085379

[409, 0.7005172103623537, 0.057636567662133556]

In [370]:
pickle.dump(ranked_cols[:fine_best_f1[0]+1], open(f"cols_selected_{sen}_v4.pickle", "wb"))

In [179]:
# Datasets que deberán predecir para la competencia.
target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}

In [197]:
df_test = pd.DataFrame()
for sen in sentiments:
    df_test = pd.concat([df_test, target[sen]], axis=0)
df_test = df_test.set_index("id")

In [198]:
pickle.dump(df_test, open("df_test.pickle", "wb"))

In [183]:
import time

In [199]:
data_all_attributes = []
times = []
for k, ix in enumerate(df_test.index):
    start_time = time.time()
    tweet = df_test.loc[ix]["tweet"]
    o = {"id": ix}
    o = {**o, **get_retro_attrib(tweet)}
    o = {**o, **get_punct_attrib(tweet)}
    o = {**o, **get_emojilib_attrib(tweet)}
    o = {**o, **get_linguistics_attrib(tweet)}
    data_all_attributes.append(o)
    dt = time.time()-start_time
    times.append(dt)
    print(f"""
    row: {k+1}, total_rows: {df_test.shape[0]}
    progress: {np.round(100*(k+1)/df_test.shape[0], 3)}%
    wait time: {datetime.timedelta(seconds = np.mean(times)*(df_test.shape[0]-k-1))}s
    """)


    row: 1, total_rows: 3142
    progress: 0.032%
    wait time: 0:02:02.688573s
    

    row: 2, total_rows: 3142
    progress: 0.064%
    wait time: 0:01:10.753806s
    

    row: 3, total_rows: 3142
    progress: 0.095%
    wait time: 0:00:54.484470s
    

    row: 4, total_rows: 3142
    progress: 0.127%
    wait time: 0:00:47.917243s
    

    row: 5, total_rows: 3142
    progress: 0.159%
    wait time: 0:00:43.345052s
    

    row: 6, total_rows: 3142
    progress: 0.191%
    wait time: 0:00:39.248250s
    

    row: 7, total_rows: 3142
    progress: 0.223%
    wait time: 0:00:35.872102s
    

    row: 8, total_rows: 3142
    progress: 0.255%
    wait time: 0:00:33.730649s
    

    row: 9, total_rows: 3142
    progress: 0.286%
    wait time: 0:00:32.063739s
    

    row: 10, total_rows: 3142
    progress: 0.318%
    wait time: 0:00:30.415609s
    

    row: 11, total_rows: 3142
    progress: 0.35%
    wait time: 0:00:28.781342s
    

    row: 12, total_rows: 3142
    progres


    row: 96, total_rows: 3142
    progress: 3.055%
    wait time: 0:00:18.896814s
    

    row: 97, total_rows: 3142
    progress: 3.087%
    wait time: 0:00:18.884386s
    

    row: 98, total_rows: 3142
    progress: 3.119%
    wait time: 0:00:18.872073s
    

    row: 99, total_rows: 3142
    progress: 3.151%
    wait time: 0:00:18.859896s
    

    row: 100, total_rows: 3142
    progress: 3.183%
    wait time: 0:00:18.817424s
    

    row: 101, total_rows: 3142
    progress: 3.215%
    wait time: 0:00:18.745530s
    

    row: 102, total_rows: 3142
    progress: 3.246%
    wait time: 0:00:18.734649s
    

    row: 103, total_rows: 3142
    progress: 3.278%
    wait time: 0:00:18.753365s
    

    row: 104, total_rows: 3142
    progress: 3.31%
    wait time: 0:00:18.683896s
    

    row: 105, total_rows: 3142
    progress: 3.342%
    wait time: 0:00:18.644602s
    

    row: 106, total_rows: 3142
    progress: 3.374%
    wait time: 0:00:18.605983s
    

    row: 107, total_rows:


    row: 215, total_rows: 3142
    progress: 6.843%
    wait time: 0:00:18.000850s
    

    row: 216, total_rows: 3142
    progress: 6.875%
    wait time: 0:00:18.006303s
    

    row: 217, total_rows: 3142
    progress: 6.906%
    wait time: 0:00:17.998149s
    

    row: 218, total_rows: 3142
    progress: 6.938%
    wait time: 0:00:17.976571s
    

    row: 219, total_rows: 3142
    progress: 6.97%
    wait time: 0:00:17.955182s
    

    row: 220, total_rows: 3142
    progress: 7.002%
    wait time: 0:00:17.907334s
    

    row: 221, total_rows: 3142
    progress: 7.034%
    wait time: 0:00:17.886349s
    

    row: 222, total_rows: 3142
    progress: 7.066%
    wait time: 0:00:17.852343s
    

    row: 223, total_rows: 3142
    progress: 7.097%
    wait time: 0:00:17.844814s
    

    row: 224, total_rows: 3142
    progress: 7.129%
    wait time: 0:00:17.837287s
    

    row: 225, total_rows: 3142
    progress: 7.161%
    wait time: 0:00:17.829789s
    

    row: 226, total_r


    row: 311, total_rows: 3142
    progress: 9.898%
    wait time: 0:00:16.974440s
    

    row: 312, total_rows: 3142
    progress: 9.93%
    wait time: 0:00:16.959453s
    

    row: 313, total_rows: 3142
    progress: 9.962%
    wait time: 0:00:16.971666s
    

    row: 314, total_rows: 3142
    progress: 9.994%
    wait time: 0:00:16.965727s
    

    row: 315, total_rows: 3142
    progress: 10.025%
    wait time: 0:00:16.950806s
    

    row: 316, total_rows: 3142
    progress: 10.057%
    wait time: 0:00:16.962797s
    

    row: 317, total_rows: 3142
    progress: 10.089%
    wait time: 0:00:16.965735s
    

    row: 318, total_rows: 3142
    progress: 10.121%
    wait time: 0:00:16.959731s
    

    row: 319, total_rows: 3142
    progress: 10.153%
    wait time: 0:00:16.962583s
    

    row: 320, total_rows: 3142
    progress: 10.185%
    wait time: 0:00:16.947705s
    

    row: 321, total_rows: 3142
    progress: 10.216%
    wait time: 0:00:16.950505s
    

    row: 322, 


    row: 434, total_rows: 3142
    progress: 13.813%
    wait time: 0:00:16.205838s
    

    row: 435, total_rows: 3142
    progress: 13.845%
    wait time: 0:00:16.199983s
    

    row: 436, total_rows: 3142
    progress: 13.877%
    wait time: 0:00:16.187916s
    

    row: 437, total_rows: 3142
    progress: 13.908%
    wait time: 0:00:16.188273s
    

    row: 438, total_rows: 3142
    progress: 13.94%
    wait time: 0:00:16.182415s
    

    row: 439, total_rows: 3142
    progress: 13.972%
    wait time: 0:00:16.195050s
    

    row: 440, total_rows: 3142
    progress: 14.004%
    wait time: 0:00:16.195292s
    

    row: 441, total_rows: 3142
    progress: 14.036%
    wait time: 0:00:16.189364s
    

    row: 442, total_rows: 3142
    progress: 14.067%
    wait time: 0:00:16.177329s
    

    row: 443, total_rows: 3142
    progress: 14.099%
    wait time: 0:00:16.159225s
    

    row: 444, total_rows: 3142
    progress: 14.131%
    wait time: 0:00:16.147268s
    

    row: 4

...


    row: 2676, total_rows: 3142
    progress: 85.169%
    wait time: 0:00:02.701531s
    

    row: 2677, total_rows: 3142
    progress: 85.201%
    wait time: 0:00:02.696117s
    

    row: 2678, total_rows: 3142
    progress: 85.232%
    wait time: 0:00:02.690355s
    

    row: 2679, total_rows: 3142
    progress: 85.264%
    wait time: 0:00:02.684939s
    

    row: 2680, total_rows: 3142
    progress: 85.296%
    wait time: 0:00:02.679003s
    

    row: 2681, total_rows: 3142
    progress: 85.328%
    wait time: 0:00:02.673067s
    

    row: 2682, total_rows: 3142
    progress: 85.36%
    wait time: 0:00:02.667647s
    

    row: 2683, total_rows: 3142
    progress: 85.391%
    wait time: 0:00:02.662226s
    

    row: 2684, total_rows: 3142
    progress: 85.423%
    wait time: 0:00:02.656290s
    

    row: 2685, total_rows: 3142
    progress: 85.455%
    wait time: 0:00:02.650696s
    

    row: 2686, total_rows: 3142
    progress: 85.487%
    wait time: 0:00:02.644591s
    



    row: 2770, total_rows: 3142
    progress: 88.16%
    wait time: 0:00:02.154914s
    

    row: 2771, total_rows: 3142
    progress: 88.192%
    wait time: 0:00:02.148881s
    

    row: 2772, total_rows: 3142
    progress: 88.224%
    wait time: 0:00:02.143118s
    

    row: 2773, total_rows: 3142
    progress: 88.256%
    wait time: 0:00:02.137221s
    

    row: 2774, total_rows: 3142
    progress: 88.288%
    wait time: 0:00:02.131324s
    

    row: 2775, total_rows: 3142
    progress: 88.32%
    wait time: 0:00:02.125296s
    

    row: 2776, total_rows: 3142
    progress: 88.351%
    wait time: 0:00:02.119269s
    

    row: 2777, total_rows: 3142
    progress: 88.383%
    wait time: 0:00:02.113376s
    

    row: 2778, total_rows: 3142
    progress: 88.415%
    wait time: 0:00:02.107614s
    

    row: 2779, total_rows: 3142
    progress: 88.447%
    wait time: 0:00:02.101721s
    

    row: 2780, total_rows: 3142
    progress: 88.479%
    wait time: 0:00:02.095959s
    




    row: 2864, total_rows: 3142
    progress: 91.152%
    wait time: 0:00:01.607684s
    

    row: 2865, total_rows: 3142
    progress: 91.184%
    wait time: 0:00:01.601729s
    

    row: 2866, total_rows: 3142
    progress: 91.216%
    wait time: 0:00:01.595871s
    

    row: 2867, total_rows: 3142
    progress: 91.248%
    wait time: 0:00:01.590111s
    

    row: 2868, total_rows: 3142
    progress: 91.279%
    wait time: 0:00:01.584445s
    

    row: 2869, total_rows: 3142
    progress: 91.311%
    wait time: 0:00:01.578779s
    

    row: 2870, total_rows: 3142
    progress: 91.343%
    wait time: 0:00:01.572922s
    

    row: 2871, total_rows: 3142
    progress: 91.375%
    wait time: 0:00:01.566971s
    

    row: 2872, total_rows: 3142
    progress: 91.407%
    wait time: 0:00:01.561398s
    

    row: 2873, total_rows: 3142
    progress: 91.439%
    wait time: 0:00:01.555636s
    

    row: 2874, total_rows: 3142
    progress: 91.47%
    wait time: 0:00:01.549781s
    



    row: 2982, total_rows: 3142
    progress: 94.908%
    wait time: 0:00:00.926210s
    

    row: 2983, total_rows: 3142
    progress: 94.94%
    wait time: 0:00:00.920540s
    

    row: 2984, total_rows: 3142
    progress: 94.971%
    wait time: 0:00:00.914709s
    

    row: 2985, total_rows: 3142
    progress: 95.003%
    wait time: 0:00:00.908983s
    

    row: 2986, total_rows: 3142
    progress: 95.035%
    wait time: 0:00:00.903205s
    

    row: 2987, total_rows: 3142
    progress: 95.067%
    wait time: 0:00:00.897323s
    

    row: 2988, total_rows: 3142
    progress: 95.099%
    wait time: 0:00:00.891441s
    

    row: 2989, total_rows: 3142
    progress: 95.13%
    wait time: 0:00:00.885561s
    

    row: 2990, total_rows: 3142
    progress: 95.162%
    wait time: 0:00:00.879683s
    

    row: 2991, total_rows: 3142
    progress: 95.194%
    wait time: 0:00:00.873957s
    

    row: 2992, total_rows: 3142
    progress: 95.226%
    wait time: 0:00:00.868230s
    




    row: 3074, total_rows: 3142
    progress: 97.836%
    wait time: 0:00:00.393549s
    

    row: 3075, total_rows: 3142
    progress: 97.868%
    wait time: 0:00:00.387744s
    

    row: 3076, total_rows: 3142
    progress: 97.899%
    wait time: 0:00:00.381940s
    

    row: 3077, total_rows: 3142
    progress: 97.931%
    wait time: 0:00:00.376158s
    

    row: 3078, total_rows: 3142
    progress: 97.963%
    wait time: 0:00:00.370375s
    

    row: 3079, total_rows: 3142
    progress: 97.995%
    wait time: 0:00:00.364593s
    

    row: 3080, total_rows: 3142
    progress: 98.027%
    wait time: 0:00:00.358770s
    

    row: 3081, total_rows: 3142
    progress: 98.059%
    wait time: 0:00:00.353007s
    

    row: 3082, total_rows: 3142
    progress: 98.09%
    wait time: 0:00:00.347244s
    

    row: 3083, total_rows: 3142
    progress: 98.122%
    wait time: 0:00:00.341442s
    

    row: 3084, total_rows: 3142
    progress: 98.154%
    wait time: 0:00:00.335658s
    


In [200]:
df_representation_test = pd.DataFrame(data_all_attributes).set_index("id").fillna(0)
df_representation_test

,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,retro<&>num_tokens_upper,retro<&>prop_vowels,retro<&>len_max_rep_char,retro<&>max_char_fre_per_token(o),retro<&>max_char_fre_per_token(s),...,linguistics<&>lemma<&>initiate,linguistics<&>lemma<&>transaction,linguistics<&>lemma<&>@aceoperative789,linguistics<&>shape<&>@XxxXxxxxddd,linguistics<&>lemma<&>afew,linguistics<&>lemma<&>audio,linguistics<&>lemma<&>supply,linguistics<&>lemma<&>acrylic,linguistics<&>lemma<&>@sainsburys,linguistics<&>lemma<&>chafford
id,,,,,,,,,,,,,,,,,,,,,
10941,21,109,0,21,1,0,0.370787,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10942,8,60,2,10,6,5,0.150943,2,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10943,23,115,0,23,4,0,0.344086,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10944,19,98,0,20,4,0,0.375000,2,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10945,21,131,3,22,4,1,0.288288,2,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41528,7,41,0,6,3,1,0.257143,2,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41529,22,132,0,23,3,2,0.324324,2,1,1,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41530,25,139,1,25,1,0,0.382609,2,1,1,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [201]:
pickle.dump(df_representation_test, open("df_representation_test_v1.pickle", "wb"))

In [209]:
data_bertweet = []
col_names = [f"BERTweet_{i}" for i in range(768)]
for k, ix in enumerate(df_test.index):
    print(k+1, df_test.shape[0], 100*(k+1)/df_test.shape[0])
    tweet = df_test.loc[ix]["tweet"]
    input_ids = torch.tensor([tokenizer.encode(tweet)])
    with torch.no_grad():
        outputs = bertweet(input_ids)
        hidden_states = outputs[0]

    token_embeddings = np.array([ll.numpy() for ll in hidden_states[0]])
    sentence_embedding = np.mean(token_embeddings, axis=0)
    o = {"id": ix}
    o = {**o, **dict(zip(col_names, sentence_embedding))}
    data_bertweet.append(o)

1 3142 0.031826861871419476
2 3142 0.06365372374283895
3 3142 0.09548058561425843
4 3142 0.1273074474856779
5 3142 0.15913430935709738
6 3142 0.19096117122851686
7 3142 0.22278803309993633
8 3142 0.2546148949713558
9 3142 0.2864417568427753
10 3142 0.31826861871419476
11 3142 0.35009548058561424
12 3142 0.3819223424570337
13 3142 0.4137492043284532
14 3142 0.44557606619987267
15 3142 0.47740292807129214
16 3142 0.5092297899427116
17 3142 0.5410566518141311
18 3142 0.5728835136855506
19 3142 0.60471037555697
20 3142 0.6365372374283895
21 3142 0.668364099299809
22 3142 0.7001909611712285
23 3142 0.732017823042648
24 3142 0.7638446849140674
25 3142 0.7956715467854869
26 3142 0.8274984086569064
27 3142 0.8593252705283259
28 3142 0.8911521323997453
29 3142 0.9229789942711648
30 3142 0.9548058561425843
31 3142 0.9866327180140039
32 3142 1.0184595798854232
33 3142 1.0502864417568427
34 3142 1.0821133036282622
35 3142 1.1139401654996817
36 3142 1.1457670273711011
37 3142 1.1775938892425206
38 

306 3142 9.739019732654361
307 3142 9.77084659452578
308 3142 9.8026734563972
309 3142 9.834500318268619
310 3142 9.866327180140038
311 3142 9.898154042011457
312 3142 9.929980903882877
313 3142 9.961807765754296
314 3142 9.993634627625717
315 3142 10.025461489497136
316 3142 10.057288351368555
317 3142 10.089115213239975
318 3142 10.120942075111394
319 3142 10.152768936982813
320 3142 10.184595798854232
321 3142 10.216422660725652
322 3142 10.248249522597073
323 3142 10.280076384468492
324 3142 10.311903246339911
325 3142 10.34373010821133
326 3142 10.37555697008275
327 3142 10.407383831954169
328 3142 10.439210693825588
329 3142 10.471037555697007
330 3142 10.502864417568428
331 3142 10.534691279439848
332 3142 10.566518141311267
333 3142 10.598345003182686
334 3142 10.630171865054105
335 3142 10.661998726925525
336 3142 10.693825588796944
337 3142 10.725652450668363
338 3142 10.757479312539784
339 3142 10.789306174411204
340 3142 10.821133036282623
341 3142 10.852959898154042
342 31

603 3142 19.191597708465945
604 3142 19.223424570337365
605 3142 19.255251432208784
606 3142 19.287078294080203
607 3142 19.318905155951622
608 3142 19.35073201782304
609 3142 19.38255887969446
610 3142 19.41438574156588
611 3142 19.446212603437303
612 3142 19.478039465308722
613 3142 19.50986632718014
614 3142 19.54169318905156
615 3142 19.57352005092298
616 3142 19.6053469127944
617 3142 19.63717377466582
618 3142 19.669000636537238
619 3142 19.700827498408657
620 3142 19.732654360280076
621 3142 19.764481222151495
622 3142 19.796308084022915
623 3142 19.828134945894334
624 3142 19.859961807765753
625 3142 19.891788669637172
626 3142 19.92361553150859
627 3142 19.955442393380014
628 3142 19.987269255251434
629 3142 20.019096117122853
630 3142 20.050922978994272
631 3142 20.08274984086569
632 3142 20.11457670273711
633 3142 20.14640356460853
634 3142 20.17823042647995
635 3142 20.21005728835137
636 3142 20.241884150222788
637 3142 20.273711012094207
638 3142 20.305537873965626
639 314

902 3142 28.70782940802037
903 3142 28.73965626989179
904 3142 28.77148313176321
905 3142 28.803309993634628
906 3142 28.835136855506047
907 3142 28.866963717377466
908 3142 28.898790579248885
909 3142 28.930617441120305
910 3142 28.962444302991724
911 3142 28.994271164863143
912 3142 29.026098026734562
913 3142 29.057924888605985
914 3142 29.089751750477404
915 3142 29.121578612348824
916 3142 29.153405474220243
917 3142 29.185232336091662
918 3142 29.21705919796308
919 3142 29.2488860598345
920 3142 29.28071292170592
921 3142 29.31253978357734
922 3142 29.34436664544876
923 3142 29.376193507320178
924 3142 29.408020369191597
925 3142 29.439847231063016
926 3142 29.471674092934435
927 3142 29.503500954805855
928 3142 29.535327816677274
929 3142 29.567154678548697
930 3142 29.598981540420116
931 3142 29.630808402291535
932 3142 29.662635264162954
933 3142 29.694462126034374
934 3142 29.726288987905793
935 3142 29.758115849777212
936 3142 29.78994271164863
937 3142 29.82176957352005
938

1198 3142 38.12858052196054
1199 3142 38.16040738383195
1200 3142 38.192234245703375
1201 3142 38.22406110757479
1202 3142 38.25588796944621
1203 3142 38.28771483131763
1204 3142 38.31954169318905
1205 3142 38.35136855506047
1206 3142 38.38319541693189
1207 3142 38.41502227880331
1208 3142 38.44684914067473
1209 3142 38.47867600254615
1210 3142 38.51050286441757
1211 3142 38.54232972628899
1212 3142 38.574156588160406
1213 3142 38.60598345003183
1214 3142 38.637810311903245
1215 3142 38.66963717377467
1216 3142 38.70146403564608
1217 3142 38.733290897517506
1218 3142 38.76511775938892
1219 3142 38.796944621260344
1220 3142 38.82877148313176
1221 3142 38.86059834500318
1222 3142 38.892425206874606
1223 3142 38.92425206874602
1224 3142 38.956078930617444
1225 3142 38.98790579248886
1226 3142 39.01973265436028
1227 3142 39.0515595162317
1228 3142 39.08338637810312
1229 3142 39.11521323997454
1230 3142 39.14704010184596
1231 3142 39.178866963717375
1232 3142 39.2106938255888
1233 3142 39.2

...

1785 3142 56.81094844048377
1786 3142 56.842775302355186
1787 3142 56.87460216422661
1788 3142 56.906429026098024
1789 3142 56.93825588796945
1790 3142 56.97008274984086
1791 3142 57.001909611712286
1792 3142 57.0337364735837
1793 3142 57.065563335455124
1794 3142 57.09739019732655
1795 3142 57.12921705919796
1796 3142 57.161043921069385
1797 3142 57.1928707829408
1798 3142 57.224697644812224
1799 3142 57.25652450668364
1800 3142 57.28835136855506
1801 3142 57.32017823042648
1802 3142 57.3520050922979
1803 3142 57.38383195416932
1804 3142 57.41565881604074
1805 3142 57.447485677912155
1806 3142 57.47931253978358
1807 3142 57.511139401654994
1808 3142 57.54296626352642
1809 3142 57.57479312539783
1810 3142 57.606619987269255
1811 3142 57.63844684914068
1812 3142 57.67027371101209
1813 3142 57.702100572883516
1814 3142 57.73392743475493
1815 3142 57.765754296626355
1816 3142 57.79758115849777
1817 3142 57.82940802036919
1818 3142 57.86123488224061
1819 3142 57.89306174411203
1820 3142 57

2078 3142 66.13621896880967
2079 3142 66.1680458306811
2080 3142 66.19987269255252
2081 3142 66.23169955442394
2082 3142 66.26352641629535
2083 3142 66.29535327816677
2084 3142 66.3271801400382
2085 3142 66.35900700190962
2086 3142 66.39083386378103
2087 3142 66.42266072565245
2088 3142 66.45448758752387
2089 3142 66.4863144493953
2090 3142 66.5181413112667
2091 3142 66.54996817313813
2092 3142 66.58179503500955
2093 3142 66.61362189688097
2094 3142 66.64544875875238
2095 3142 66.6772756206238
2096 3142 66.70910248249523
2097 3142 66.74092934436665
2098 3142 66.77275620623807
2099 3142 66.80458306810948
2100 3142 66.8364099299809
2101 3142 66.86823679185233
2102 3142 66.90006365372375
2103 3142 66.93189051559516
2104 3142 66.96371737746658
2105 3142 66.995544239338
2106 3142 67.02737110120943
2107 3142 67.05919796308083
2108 3142 67.09102482495226
2109 3142 67.12285168682368
2110 3142 67.1546785486951
2111 3142 67.18650541056653
2112 3142 67.21833227243793
2113 3142 67.25015913430936
2

2375 3142 75.58879694462127
2376 3142 75.62062380649267
2377 3142 75.6524506683641
2378 3142 75.68427753023552
2379 3142 75.71610439210694
2380 3142 75.74793125397835
2381 3142 75.77975811584977
2382 3142 75.8115849777212
2383 3142 75.84341183959262
2384 3142 75.87523870146404
2385 3142 75.90706556333545
2386 3142 75.93889242520687
2387 3142 75.9707192870783
2388 3142 76.00254614894972
2389 3142 76.03437301082113
2390 3142 76.06619987269255
2391 3142 76.09802673456397
2392 3142 76.1298535964354
2393 3142 76.1616804583068
2394 3142 76.19350732017823
2395 3142 76.22533418204965
2396 3142 76.25716104392107
2397 3142 76.2889879057925
2398 3142 76.3208147676639
2399 3142 76.35264162953533
2400 3142 76.38446849140675
2401 3142 76.41629535327817
2402 3142 76.44812221514958
2403 3142 76.479949077021
2404 3142 76.51177593889243
2405 3142 76.54360280076385
2406 3142 76.57542966263526
2407 3142 76.60725652450668
2408 3142 76.6390833863781
2409 3142 76.67091024824953
2410 3142 76.70273711012094
24

2673 3142 85.07320178230427
2674 3142 85.10502864417569
2675 3142 85.1368555060471
2676 3142 85.16868236791852
2677 3142 85.20050922978994
2678 3142 85.23233609166137
2679 3142 85.26416295353278
2680 3142 85.2959898154042
2681 3142 85.32781667727562
2682 3142 85.35964353914704
2683 3142 85.39147040101847
2684 3142 85.42329726288988
2685 3142 85.4551241247613
2686 3142 85.48695098663272
2687 3142 85.51877784850414
2688 3142 85.55060471037555
2689 3142 85.58243157224697
2690 3142 85.6142584341184
2691 3142 85.64608529598982
2692 3142 85.67791215786123
2693 3142 85.70973901973265
2694 3142 85.74156588160407
2695 3142 85.7733927434755
2696 3142 85.8052196053469
2697 3142 85.83704646721833
2698 3142 85.86887332908975
2699 3142 85.90070019096117
2700 3142 85.9325270528326
2701 3142 85.964353914704
2702 3142 85.99618077657543
2703 3142 86.02800763844685
2704 3142 86.05983450031827
2705 3142 86.09166136218968
2706 3142 86.1234882240611
2707 3142 86.15531508593253
2708 3142 86.18714194780395
27

2968 3142 94.46212603437301
2969 3142 94.49395289624444
2970 3142 94.52577975811585
2971 3142 94.55760661998727
2972 3142 94.58943348185869
2973 3142 94.62126034373011
2974 3142 94.65308720560152
2975 3142 94.68491406747295
2976 3142 94.71674092934437
2977 3142 94.74856779121579
2978 3142 94.7803946530872
2979 3142 94.81222151495862
2980 3142 94.84404837683005
2981 3142 94.87587523870147
2982 3142 94.90770210057288
2983 3142 94.9395289624443
2984 3142 94.97135582431572
2985 3142 95.00318268618715
2986 3142 95.03500954805857
2987 3142 95.06683640992998
2988 3142 95.0986632718014
2989 3142 95.13049013367282
2990 3142 95.16231699554424
2991 3142 95.19414385741565
2992 3142 95.22597071928708
2993 3142 95.2577975811585
2994 3142 95.28962444302992
2995 3142 95.32145130490133
2996 3142 95.35327816677275
2997 3142 95.38510502864418
2998 3142 95.4169318905156
2999 3142 95.44875875238701
3000 3142 95.48058561425843
3001 3142 95.51241247612985
3002 3142 95.54423933800128
3003 3142 95.576066199872

In [214]:
pickle.dump(pd.DataFrame(data_bertweet).set_index("id"), open("df_representation_test_v2.pickle", "wb"))
df_representation_test_v2 = pickle.load(open("df_representation_test_v2.pickle", "rb"))

In [215]:
df_representation_test_v3 = df_representation_test
df_representation_test_v3 = pd.concat([df_representation_test_v3, df_representation_test_v2], axis=1)
df_representation_test_v3

,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,retro<&>num_tokens_upper,retro<&>prop_vowels,retro<&>len_max_rep_char,retro<&>max_char_fre_per_token(o),retro<&>max_char_fre_per_token(s),...,BERTweet_758,BERTweet_759,BERTweet_760,BERTweet_761,BERTweet_762,BERTweet_763,BERTweet_764,BERTweet_765,BERTweet_766,BERTweet_767
id,,,,,,,,,,,,,,,,,,,,,
10941,21,109,0,21,1,0,0.370787,2,2,2,...,0.123334,0.062934,0.063844,-0.114385,-0.223249,-0.082935,0.096100,0.134355,-0.089524,0.052725
10942,8,60,2,10,6,5,0.150943,2,3,2,...,0.114743,-0.102170,0.210606,0.015215,-0.190899,-0.090932,-0.051202,0.083870,0.000473,0.243169
10943,23,115,0,23,4,0,0.344086,2,2,2,...,-0.032676,0.090184,0.130342,-0.079150,-0.150815,0.017163,0.346057,-0.046520,-0.133298,-0.233223
10944,19,98,0,20,4,0,0.375000,2,1,2,...,-0.017395,-0.029215,-0.080877,0.040828,0.020043,-0.034798,0.240827,0.083698,0.198686,0.055953
10945,21,131,3,22,4,1,0.288288,2,3,2,...,0.052519,-0.201800,-0.094245,0.137064,-0.070205,-0.114456,0.057291,0.062598,0.147984,-0.147572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41528,7,41,0,6,3,1,0.257143,2,1,1,...,-0.070892,-0.112746,0.078887,0.042544,0.017418,-0.242335,0.189593,0.099153,0.250478,0.002542
41529,22,132,0,23,3,2,0.324324,2,1,1,...,0.145892,-0.139355,0.015763,0.104158,0.099488,0.047747,0.095338,-0.118079,0.227708,-0.134026
41530,25,139,1,25,1,0,0.382609,2,1,1,...,0.093361,-0.109361,0.008455,0.067768,-0.013469,-0.081498,-0.004565,0.038196,-0.042650,-0.140450


In [216]:
pickle.dump(df_representation_test_v3, open("df_representation_test_v3.pickle", "wb"))

In [223]:
data_lexicon_attributes = []
times = []
for k, ix in enumerate(df_test.index):
    start_time = time.time()
    tweet = df_test.loc[ix]["tweet"]
    o = {"id": ix}
    o = {**o, **get_lexicon_attrib(tweet)}
    data_lexicon_attributes.append(o)
    dt = time.time()-start_time
    times.append(dt)
    print(f"""
    row: {k+1}, total_rows: {df_test.shape[0]}
    progress: {np.round(100*(k+1)/df_test.shape[0], 3)}%
    wait time: {datetime.timedelta(seconds = np.median(times)*(df_test.shape[0]-k-1))}s
    """)


    row: 1, total_rows: 3142
    progress: 0.032%
    wait time: 0:13:16.259178s
    

    row: 2, total_rows: 3142
    progress: 0.064%
    wait time: 0:09:13.576164s
    

    row: 3, total_rows: 3142
    progress: 0.095%
    wait time: 0:13:15.752168s
    

    row: 4, total_rows: 3142
    progress: 0.127%
    wait time: 0:12:43.573287s
    

    row: 5, total_rows: 3142
    progress: 0.159%
    wait time: 0:13:11.243043s
    

    row: 6, total_rows: 3142
    progress: 0.191%
    wait time: 0:13:12.991234s
    

    row: 7, total_rows: 3142
    progress: 0.223%
    wait time: 0:13:10.738585s
    

    row: 8, total_rows: 3142
    progress: 0.255%
    wait time: 0:12:46.302730s
    

    row: 9, total_rows: 3142
    progress: 0.286%
    wait time: 0:12:21.882309s
    

    row: 10, total_rows: 3142
    progress: 0.318%
    wait time: 0:12:15.947239s
    

    row: 11, total_rows: 3142
    progress: 0.35%
    wait time: 0:12:10.015808s
    

    row: 12, total_rows: 3142
    progres


    row: 95, total_rows: 3142
    progress: 3.024%
    wait time: 0:10:06.562587s
    

    row: 96, total_rows: 3142
    progress: 3.055%
    wait time: 0:10:12.631912s
    

    row: 97, total_rows: 3142
    progress: 3.087%
    wait time: 0:10:18.697121s
    

    row: 98, total_rows: 3142
    progress: 3.119%
    wait time: 0:10:20.019092s
    

    row: 99, total_rows: 3142
    progress: 3.151%
    wait time: 0:10:18.290752s
    

    row: 100, total_rows: 3142
    progress: 3.183%
    wait time: 0:10:11.827405s
    

    row: 101, total_rows: 3142
    progress: 3.215%
    wait time: 0:10:05.368174s
    

    row: 102, total_rows: 3142
    progress: 3.246%
    wait time: 0:10:11.425152s
    

    row: 103, total_rows: 3142
    progress: 3.278%
    wait time: 0:10:17.478014s
    

    row: 104, total_rows: 3142
    progress: 3.31%
    wait time: 0:10:11.022898s
    

    row: 105, total_rows: 3142
    progress: 3.342%
    wait time: 0:10:04.571899s
    

    row: 106, total_rows: 


    row: 188, total_rows: 3142
    progress: 5.983%
    wait time: 0:10:50.469574s
    

    row: 189, total_rows: 3142
    progress: 6.015%
    wait time: 0:10:47.292364s
    

    row: 190, total_rows: 3142
    progress: 6.047%
    wait time: 0:10:45.597625s
    

    row: 191, total_rows: 3142
    progress: 6.079%
    wait time: 0:10:43.903885s
    

    row: 192, total_rows: 3142
    progress: 6.111%
    wait time: 0:10:36.305946s
    

    row: 193, total_rows: 3142
    progress: 6.143%
    wait time: 0:10:28.713010s
    

    row: 194, total_rows: 3142
    progress: 6.174%
    wait time: 0:10:25.382288s
    

    row: 195, total_rows: 3142
    progress: 6.206%
    wait time: 0:10:28.286620s
    

    row: 196, total_rows: 3142
    progress: 6.238%
    wait time: 0:10:35.443158s
    

    row: 197, total_rows: 3142
    progress: 6.27%
    wait time: 0:10:27.860229s
    

    row: 198, total_rows: 3142
    progress: 6.302%
    wait time: 0:10:27.644226s
    

    row: 199, total_r


    row: 281, total_rows: 3142
    progress: 8.943%
    wait time: 0:09:38.447633s
    

    row: 282, total_rows: 3142
    progress: 8.975%
    wait time: 0:09:39.676709s
    

    row: 283, total_rows: 3142
    progress: 9.007%
    wait time: 0:09:38.043265s
    

    row: 284, total_rows: 3142
    progress: 9.039%
    wait time: 0:09:39.271341s
    

    row: 285, total_rows: 3142
    progress: 9.071%
    wait time: 0:09:37.638898s
    

    row: 286, total_rows: 3142
    progress: 9.102%
    wait time: 0:09:32.988582s
    

    row: 287, total_rows: 3142
    progress: 9.134%
    wait time: 0:09:37.234530s
    

    row: 288, total_rows: 3142
    progress: 9.166%
    wait time: 0:09:32.587329s
    

    row: 289, total_rows: 3142
    progress: 9.198%
    wait time: 0:09:36.830163s
    

    row: 290, total_rows: 3142
    progress: 9.23%
    wait time: 0:09:32.186077s
    

    row: 291, total_rows: 3142
    progress: 9.262%
    wait time: 0:09:36.425796s
    

    row: 292, total_r


    row: 373, total_rows: 3142
    progress: 11.871%
    wait time: 0:09:33.711192s
    

    row: 374, total_rows: 3142
    progress: 11.903%
    wait time: 0:09:33.507301s
    

    row: 375, total_rows: 3142
    progress: 11.935%
    wait time: 0:09:33.303408s
    

    row: 376, total_rows: 3142
    progress: 11.967%
    wait time: 0:09:33.092917s
    

    row: 377, total_rows: 3142
    progress: 11.999%
    wait time: 0:09:32.882429s
    

    row: 378, total_rows: 3142
    progress: 12.031%
    wait time: 0:09:32.678534s
    

    row: 379, total_rows: 3142
    progress: 12.062%
    wait time: 0:09:32.474635s
    

    row: 380, total_rows: 3142
    progress: 12.094%
    wait time: 0:09:33.641428s
    

    row: 381, total_rows: 3142
    progress: 12.126%
    wait time: 0:09:34.807225s
    

    row: 382, total_rows: 3142
    progress: 12.158%
    wait time: 0:09:35.979595s
    

    row: 383, total_rows: 3142
    progress: 12.19%
    wait time: 0:09:34.390849s
    

    row: 3

...


    row: 2751, total_rows: 3142
    progress: 87.556%
    wait time: 0:01:18.662491s
    

    row: 2752, total_rows: 3142
    progress: 87.588%
    wait time: 0:01:18.266137s
    

    row: 2753, total_rows: 3142
    progress: 87.619%
    wait time: 0:01:18.260126s
    

    row: 2754, total_rows: 3142
    progress: 87.651%
    wait time: 0:01:18.058943s
    

    row: 2755, total_rows: 3142
    progress: 87.683%
    wait time: 0:01:17.857760s
    

    row: 2756, total_rows: 3142
    progress: 87.715%
    wait time: 0:01:17.463407s
    

    row: 2757, total_rows: 3142
    progress: 87.747%
    wait time: 0:01:17.070055s
    

    row: 2758, total_rows: 3142
    progress: 87.778%
    wait time: 0:01:16.869873s
    

    row: 2759, total_rows: 3142
    progress: 87.81%
    wait time: 0:01:16.669691s
    

    row: 2760, total_rows: 3142
    progress: 87.842%
    wait time: 0:01:16.469509s
    

    row: 2761, total_rows: 3142
    progress: 87.874%
    wait time: 0:01:16.269327s
    



    row: 2844, total_rows: 3142
    progress: 90.516%
    wait time: 0:00:59.654153s
    

    row: 2845, total_rows: 3142
    progress: 90.547%
    wait time: 0:00:59.453901s
    

    row: 2846, total_rows: 3142
    progress: 90.579%
    wait time: 0:00:59.253790s
    

    row: 2847, total_rows: 3142
    progress: 90.611%
    wait time: 0:00:59.053679s
    

    row: 2848, total_rows: 3142
    progress: 90.643%
    wait time: 0:00:58.853497s
    

    row: 2849, total_rows: 3142
    progress: 90.675%
    wait time: 0:00:58.653315s
    

    row: 2850, total_rows: 3142
    progress: 90.707%
    wait time: 0:00:58.599261s
    

    row: 2851, total_rows: 3142
    progress: 90.738%
    wait time: 0:00:58.252951s
    

    row: 2852, total_rows: 3142
    progress: 90.77%
    wait time: 0:00:58.197896s
    

    row: 2853, total_rows: 3142
    progress: 90.802%
    wait time: 0:00:57.852587s
    

    row: 2854, total_rows: 3142
    progress: 90.834%
    wait time: 0:00:57.796532s
    



    row: 2936, total_rows: 3142
    progress: 93.444%
    wait time: 0:00:41.649822s
    

    row: 2937, total_rows: 3142
    progress: 93.475%
    wait time: 0:00:41.447614s
    

    row: 2938, total_rows: 3142
    progress: 93.507%
    wait time: 0:00:41.245431s
    

    row: 2939, total_rows: 3142
    progress: 93.539%
    wait time: 0:00:41.043247s
    

    row: 2940, total_rows: 3142
    progress: 93.571%
    wait time: 0:00:40.841064s
    

    row: 2941, total_rows: 3142
    progress: 93.603%
    wait time: 0:00:40.638880s
    

    row: 2942, total_rows: 3142
    progress: 93.635%
    wait time: 0:00:40.436697s
    

    row: 2943, total_rows: 3142
    progress: 93.666%
    wait time: 0:00:40.234514s
    

    row: 2944, total_rows: 3142
    progress: 93.698%
    wait time: 0:00:40.032330s
    

    row: 2945, total_rows: 3142
    progress: 93.73%
    wait time: 0:00:39.830147s
    

    row: 2946, total_rows: 3142
    progress: 93.762%
    wait time: 0:00:39.627963s
    



    row: 3027, total_rows: 3142
    progress: 96.34%
    wait time: 0:00:23.136027s
    

    row: 3028, total_rows: 3142
    progress: 96.372%
    wait time: 0:00:22.877794s
    

    row: 3029, total_rows: 3142
    progress: 96.404%
    wait time: 0:00:22.620562s
    

    row: 3030, total_rows: 3142
    progress: 96.435%
    wait time: 0:00:22.476429s
    

    row: 3031, total_rows: 3142
    progress: 96.467%
    wait time: 0:00:22.331295s
    

    row: 3032, total_rows: 3142
    progress: 96.499%
    wait time: 0:00:22.075064s
    

    row: 3033, total_rows: 3142
    progress: 96.531%
    wait time: 0:00:21.819834s
    

    row: 3034, total_rows: 3142
    progress: 96.563%
    wait time: 0:00:21.619652s
    

    row: 3035, total_rows: 3142
    progress: 96.595%
    wait time: 0:00:21.419470s
    

    row: 3036, total_rows: 3142
    progress: 96.626%
    wait time: 0:00:21.272335s
    

    row: 3037, total_rows: 3142
    progress: 96.658%
    wait time: 0:00:21.019106s
    



    row: 3119, total_rows: 3142
    progress: 99.268%
    wait time: 0:00:04.604185s
    

    row: 3120, total_rows: 3142
    progress: 99.3%
    wait time: 0:00:04.403998s
    

    row: 3121, total_rows: 3142
    progress: 99.332%
    wait time: 0:00:04.203811s
    

    row: 3122, total_rows: 3142
    progress: 99.363%
    wait time: 0:00:04.003634s
    

    row: 3123, total_rows: 3142
    progress: 99.395%
    wait time: 0:00:03.803457s
    

    row: 3124, total_rows: 3142
    progress: 99.427%
    wait time: 0:00:03.603275s
    

    row: 3125, total_rows: 3142
    progress: 99.459%
    wait time: 0:00:03.403093s
    

    row: 3126, total_rows: 3142
    progress: 99.491%
    wait time: 0:00:03.210918s
    

    row: 3127, total_rows: 3142
    progress: 99.523%
    wait time: 0:00:03.002729s
    

    row: 3128, total_rows: 3142
    progress: 99.554%
    wait time: 0:00:02.809554s
    

    row: 3129, total_rows: 3142
    progress: 99.586%
    wait time: 0:00:02.602365s
    



In [224]:
df_representation_test_v4 = pd.DataFrame(data_lexicon_attributes).set_index("id")
df_representation_test_v4 = pd.concat([df_representation_test_v4, df_representation_test_v3], axis=1)
df_representation_test_v4

,lexicon<&>LiuHu<&>+,lexicon<&>LiuHu<&>-,lexicon<&>sentiwordnet<&>+,lexicon<&>sentiwordnet<&>-,lexicon<&>sentiwordnet<&>o,retro<&>num_tokens,retro<&>lenght,retro<&>num_numbs,retro<&>num_alpha,retro<&>num_with_uppercase,...,BERTweet_758,BERTweet_759,BERTweet_760,BERTweet_761,BERTweet_762,BERTweet_763,BERTweet_764,BERTweet_765,BERTweet_766,BERTweet_767
id,,,,,,,,,,,,,,,,,,,,,
10941,0,0,0.750,0.250,6.000,21,109,0,21,1,...,0.123334,0.062934,0.063844,-0.114385,-0.223249,-0.082935,0.096100,0.134355,-0.089524,0.052725
10942,0,0,0.000,0.000,1.000,8,60,2,10,6,...,0.114743,-0.102170,0.210606,0.015215,-0.190899,-0.090932,-0.051202,0.083870,0.000473,0.243169
10943,0,0,0.000,0.625,5.375,23,115,0,23,4,...,-0.032676,0.090184,0.130342,-0.079150,-0.150815,0.017163,0.346057,-0.046520,-0.133298,-0.233223
10944,1,1,0.000,0.625,2.375,19,98,0,20,4,...,-0.017395,-0.029215,-0.080877,0.040828,0.020043,-0.034798,0.240827,0.083698,0.198686,0.055953
10945,0,0,0.000,0.000,2.000,21,131,3,22,4,...,0.052519,-0.201800,-0.094245,0.137064,-0.070205,-0.114456,0.057291,0.062598,0.147984,-0.147572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41528,0,1,0.125,0.250,0.625,7,41,0,6,3,...,-0.070892,-0.112746,0.078887,0.042544,0.017418,-0.242335,0.189593,0.099153,0.250478,0.002542
41529,1,0,0.000,0.125,3.875,22,132,0,23,3,...,0.145892,-0.139355,0.015763,0.104158,0.099488,0.047747,0.095338,-0.118079,0.227708,-0.134026
41530,0,1,0.875,1.000,7.125,25,139,1,25,1,...,0.093361,-0.109361,0.008455,0.067768,-0.013469,-0.081498,-0.004565,0.038196,-0.042650,-0.140450


In [225]:
pickle.dump(df_representation_test_v4, open("df_representation_test_v4.pickle", "wb"))